In [1]:
#Las librerias clasicas
import numpy as np
import pandas as pd

# Estas son las que corresponden a dash y plotly 
import dash 
from dash import dcc
from dash import html
from dash.dependencies import Input, Output, State
from dash import dash_table
import dash_bootstrap_components as dbc

import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots
import plotly.io as pio

# Cargamos archivos

In [2]:
nat_rmsd = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["nat_ncst","nat_psn1","nat_aph1","nat_pen2","toda_nat"]]
nat3_rmsd = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["nat3_ncst","nat3_psn1","nat3_aph1","nat3_pen2","toda_nat3"]]
cpx_rmsd = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["cpx_ncst","cpx_psn1","cpx_aph1","cpx_pen2","cpx_sus","toda_cpx"]]
cpx3_rmsd = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["cpx2_ncst","cpx2_psn1","cpx2_aph1","cpx2_pen2","cpx2_sus","toda_cpx2"]]

sus_rmsd = [np.genfromtxt("assets/rmsd_"+i+".dat") for i in ["sus_ncst","sus_psn1","sus_aph1","sus_pen2","sus_sus","toda_sus"]]

todo = pd.read_csv("assets_viejos/ligandoschidos.csv")

df  = pd.read_csv('assets_viejos/numerado.csv')
aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida"].values.astype("str")


nat_rmsf = np.genfromtxt('assets_viejos/rmsf_nat.dat')
gly_rmsf = np.genfromtxt('assets_viejos/rmsf_gly.dat')
gly_rmsf = np.concatenate([gly_rmsf[0:668],gly_rmsf[689:1343]])
sus_rmsf = np.genfromtxt('assets_viejos/rmsf_sus.dat')
cpx_rmsf = np.genfromtxt('assets_viejos/rmsf_cpx.dat')

#Contactos
apo_cont = np.loadtxt("assets_viejos/contactos_apo.csv", delimiter=",")
mut_cont = np.loadtxt("assets_viejos/contactos_l166p.csv",delimiter=",")
helix_df  = pd.read_csv("assets_viejos/helix_contactos.csv")

#Ubicaciones especificas
asp = pd.read_csv("assets_viejos/mut_dist_6ct_asp.dat",sep="\s+")
p31 = pd.read_csv("assets_viejos/mut_dist_6ct_p31.dat",sep="\s+")
popc = pd.read_csv("assets_viejos/mut_dist_6ct_popc.dat",sep="\s+")
tm2  = pd.read_csv("assets_viejos/mut_dist_6ct_tm2-3.dat",sep="\s+")

asp1 = pd.read_csv("assets_viejos/apo_dist_6ct_asp.dat",sep="\s+")
p311 = pd.read_csv("assets_viejos/apo_dist_6ct_p31.dat",sep="\s+")
popc1 = pd.read_csv("assets_viejos/apo_dist_6ct_popc.dat",sep="\s+")
tm21  = pd.read_csv("assets_viejos/apo_dist_6ct_tm2-3.dat",sep="\s+")

difz31 = np.loadtxt("assets_viejos/apo_difz_p31_6ct.csv", delimiter=",")
difzpopc = np.loadtxt("assets_viejos/apo_difz_centromem_6ct.csv", delimiter=",")
difz311 = np.loadtxt("assets_viejos/mut_difz_p31_6ct.csv", delimiter=",")
difzpopc1 = np.loadtxt("assets_viejos/mut_difz_centromem_6ct.csv", delimiter=",")

#distancias asparticos
nat_dist_d = np.genfromtxt('assets/asp_dist_nat.dat')
nat3_dist_d = np.genfromtxt('assets/asp_dist_nat3.dat')

sus_dist_d = np.genfromtxt('assets/asp_dist_sus.dat')
cpx_dist_d = np.genfromtxt('assets/asp_dist_cpx.dat')
cpx3_dist_d = np.genfromtxt('assets/asp_dist_cpx2.dat')

#Distancias E2012
didi = pd.read_csv("assets_viejos/e2012_dist.csv")
al =  np.load("assets_viejos/aminoacidos_distacias_all_e2012_min.npy")
aa_min2 = np.genfromtxt('assets_viejos/aminoacidos_interaccion_all_e2012_min.csv',delimiter=",",dtype="str")


#PERMM
#E = pd.read_csv("/home/chemachiles/OneDrive/ligandos/AnalisisLogP/Fenilimidazol/energias_completas.csv")
#P = pd.read_csv("/home/chemachiles/OneDrive/ligandos/AnalisisLogP/Fenilimidazol/logp_completas.csv")
#D = pd.read_csv("/home/chemachiles/OneDrive/ligandos/GSIs_GSMs.csv")


#umbrella = np.loadtxt("/home/chemachiles/Downloads/ligandos/Umbrella_sampling/fenilimidazol/wham_run/overall_perm/full_profile_perm/free_energy_profile.parse.dat")
pos_gzr = np.loadtxt("assets_viejos/MD_normal_Z_e2012.csv",delimiter=",")
#permm_imi = pd.read_csv("/home/chemachiles/Downloads/ligandos/Umbrella_sampling/combined_energies/prueba.csv",delimiter=";")
#permm_imi["Suma"]=permm_imi["Piperidona"].values+permm_imi["Fenilimidazol"].values

In [3]:
def anchomem(dd):
    extremos = dd[:,0][(dd[:,0] >= -21.5) & (dd[:,0] <= 21.5)][0], dd[:,0][(dd[:,0] >= -21.5) & (dd[:,0] <= 21.5)][-1]
    medios = dd[:,0][(dd[:,0] >= -18.5) & (dd[:,0] <= 18.5)][0], dd[:,0][(dd[:,0] >= -18.5) & (dd[:,0] <= 18.5)][-1]

    extremos = np.where(dd[:,0]==extremos[0])[0][0], np.where(dd[:,0]==extremos[1])[0][0]
    medios = np.where(dd[:,0]==medios[0])[0][0], np.where(dd[:,0]==medios[1])[0][0]

    res1 = dd[:,7][extremos[0]:medios[0]]-dd[:,9][extremos[0]:medios[0]]
    sumaleid1  = np.where(res1 == res1[min(range(len(res1)), key = lambda i: abs(res1[i]-0))])[0][0]
    inter1 = np.abs(dd[:,0][extremos[0]+sumaleid1])

    res2 = dd[:,7][medios[1]:extremos[1]]-dd[:,9][medios[1]:extremos[1]]
    sumaleid2  = np.where(res2 == res2[min(range(len(res2)), key = lambda i: abs(res2[i]-0))])[0][0]
    inter2 = np.abs(dd[:,0][medios[1]+sumaleid2])
    mem_ancho = inter1+inter2
    return str(mem_ancho)

# Esqueleto del dash
## Barside
[Para ver cositas del sidebar](https://stackoverflow.com/questions/62732631/how-to-collapsed-sidebar-in-dash-plotly-dash-bootstrap-components)

In [4]:
# F O R M A T O   D E   L A   B A R R A  Y  C O N T E N I D O

# styling the sidebar
SIDEBAR_STYLE = {
    "position": "fixed",  #Se mantiene aunque tu hagas mas chica la ventana
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "22rem",     #Ancho del sidebar 
    "padding": "2rem 1rem", #Cuanto de espacio hay arriba antes de que empiece el texto y el espacio que debe quedar a la derecha
    "background-color": "#f8f9fa",
}

SIDEBAR_HIDEN = {
    "position": "fixed",
    "top": 62.5,
    "left": "-16rem",
    "bottom": 0,
    "width": "16rem",
    "height": "100%",
    "z-index": 1,
    "overflow-x": "hidden",
    "transition": "all 0.5s",
    "padding": "0rem 0rem",
    "background-color": "#f8f9fa",
}

# padding for the page content
CONTENT_STYLE = {
    "margin-left": "25rem",  #Cuanto deja de espacio dsde TODA la pantalla
    "margin-right": "0rem",  #Margen a la dercha
    "padding": "0rem 0rem",  #Espacio arriba y a la derecha
}

CONTENT_STYLE1 = {
    "transition": "margin-left .5s",
    "margin-left": "2rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}
sidebar = html.Div(
    [
        dcc.Markdown("""**Selecciona el el analisis que quieres ver**"""),
        html.Hr(),
        dbc.Nav(
            [
                dbc.NavLink("Introducción", href="/", active="exact"),
                dbc.NavLink("RMSD", href="/rmsd", active="exact"),
                dbc.NavLink("RMSF", href="/rmsf", active="exact"),
                dbc.NavLink("Analisis de Estructura Secundaria", href="/sec-estruc", active="exact"),
                dbc.NavLink("HelAnal", href="/helanal", active="exact"),
                dbc.NavLink("Contactos membranales", href="/contact", active="exact"),
                dbc.NavLink("Ubicación de regiones", href="/ubicacion", active="exact"),
                dbc.NavLink("Componentes principales", href="/pca", active="exact"),
                dbc.NavLink("Cross-Correlation", href="/cross", active="exact"),
                dbc.NavLink("Accesibilidad al solvente", href="/h2o", active="exact"),
                dbc.NavLink("Asparticos cataliticos", href="/asp-cat", active="exact"),
                dbc.NavLink("Integridad membrana", href="/lipyphilic", active="exact"),
                dbc.NavLink("Ubicación del modulador", href="/e2012", active="exact"),
                dbc.NavLink("Analisis de Ligandos", href="/lig-imi", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    id="sidebar",
    style=SIDEBAR_STYLE,
)

content = html.Div(id="page-content", children=[], style=CONTENT_STYLE)


## Encabezado

In [5]:
encabezado  = html.Div([
    dbc.Row(dbc.Button("Sidebar", outline=True, color="secondary", className="mr-1", id="btn_sidebar")),
    dbc.Row([
    
    dbc.Col(html.A(html.Img(src="assets/UNAM.png",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center"),
                   href="https://www.unam.mx/",target='_blank'), 
            width={"size":2,"offset":0}),

    dbc.Col(html.H2("Análisis y-Secretasa",style={"color":"#8a070d"},className="text-center" "mb-4" "h50"),
            width={"size":4,"offset":2}),

    dbc.Col(html.A(html.Img(src="assets/mdcbq.svg",style={'height':'150%', 'width':'110%'},className="d-flex align-self-center"),
                   href="https://www.mdcbq.posgrado.unam.mx/home",target='_blank'),
            width={"size":2,"offset":1})
],justify="center", align="center")])

## Introducción y RMSD

In [6]:
Intro = html.Div([
    html.H4("Introducción"),
    html.P([html.Span("Esta página fue diseñada mediante dash-plotly y otras librerias de python, tiene como objetivo ayudar a explorar los resultados obtenidos por dinámica molecuar para el estudio de y-Secretasa, de una manera interactiva. En esta seccion encontraras una breve descripción de los objetos con los cuales podras interactuar.")]),
    html.Span("A lo largo de las paginas las instrucciónes se marcan en azul",style={"color":"#052080"}),
    html.H4("Objetos interactivos"),
    html.H6("Gráficos"),
    html.P("Diseñamos los gráficos para analizar los datos de una forma rápida y fácil. Ten en cuenta que si colocas el cursor sobre un punto, se mostrará la información asociada, y si vas a la esquina superior derecha del gráfico, encontrarás una barra de herramientas, que te permitira desplazarte dentro de la figura o descargar la imagen. Los graficos se pueden descargar como figuras estaticas. En el siguiente menu desplegable podras escoger el tipo de formato en el cual se descargaran las imagenes (de manera predeterminada esta png). "),
    html.Br(),
    dbc.Row([
        dbc.Col(html.Span("Escoge el formato en cual quieres descargar tus plots",style={"color":"#052080"})),
        dbc.Col(dcc.Dropdown(
            id="format-dw-img",
            options=[{'label': 'svg', 'value': 'svg'},
                     {'label': 'png', 'value': 'png'},
                     {'label': 'jpeg', 'value': 'jpeg'},
                     {'label': 'webp', 'value': 'webp'}],
            value='png',style={'width':"45%"}))],
        justify="center", align="center"),
])

p_container = html.Div([
    html.H4("Introducción"),
    html.P("La desviación cuadrática media o RMSD por sus siglas en inglés es una medida de uso común cuando el objetivo se centra en comparar la distancia o similitud conformacional de dos estructuras; o de otro modo analizar cambios conformacionales de una sola estructura a lo largo del tiempo. Para calcular el RMSD es necesario realizar dos pasos importantes 1) Paso de alineación: en donde se establece una relación uno a uno de los átomos equivalentes de ambas estructuras 2) Paso de superposición óptima: en donde se rota y traslada la estructura de manera que se minimice la suma ponderada de los cuadrados de las distancias de cada uno de los átomos56. En casos de dinámica molecular, en donde se requiere analizar la similitud conformacional a lo largo del tiempo, se puede expresar el cálculo del RMSD como:"),
    dbc.Row([dcc.Markdown('$RMSD(x,x^{ref}) =\\sqrt{\\frac{1}{N} \\sum_{i=1}^{N}[(R \\cdot x_{i}(t)+τ)-x_{i}^{ref}]^{2}}$',mathjax=True)],justify="center",),
    html.P("Que básicamente representa la suma de las distancias euclidianas de cada uno de los átomos, entre una estructura xref y otra estructura x cuando se le aplica un factor de Rotación R y traslación τ que minimizan estas distancias."),
    html.P("Generalmente, los cálculos de RMSD se realizan utilizando únicamente el esqueleto de la proteína, ya que los cambios conformacionales de las cadenas laterales de cada uno de los residuos no son representativos de un cambio conformacional general."),
    html.Br(),
    dbc.Row([
        dbc.Col([html.Span("Observemos los graficos por:  ",style={'display':'block'}),
                 dcc.Dropdown(
                    id="tipormsd",
                    options=[{'label': 'Por sistema', 'value': 'sis'},
                             {'label': 'Por subunidad', 'value': 'comp'}],
                    value='comp',style={'width':"70%",'display':'block'})]),
        dbc.Col([html.Span("Sistema: ",style={'display':'block'}),
                 dcc.Dropdown(
                    id="estruc",
                    options=[{'label': 'Estructura Nativa', 'value': 'nat'},
                             {'label': 'Estructura Nativa3', 'value': 'gly'},
                             {'label': 'Complejo Enzima-Sustrato', 'value': 'sus'},
                             {'label': 'Complejo Enzima-Sustrato-Modulador', 'value': 'cpx'},
                            {'label': 'Complejo Enzima-Sustrato-Modulador2', 'value': 'mut'},],
                    value='nat',style={'width':"80%",'display':'block'})]),
        dbc.Col([html.Span("Subunidad: ",style={'display':'block'}),
                 dcc.Dropdown(
                    id="sub1",
                    options=[{'label': 'NCST', 'value': 'ncst'},
                             {'label': 'PSN1', 'value': 'psn1'},
                             {'label': 'APH1', 'value': 'aph1'},
                             {'label': 'PEN2', 'value': 'pen2'},
                             {'label': 'C83', 'value': 'sus'},
                             {'label': 'Toda', 'value': 'all'}],
                    value='psn1',style={'width':"55%",'display':'block'})])]),
    
    html.Div(id="plots-rmsd"),
])

## RMSF y Estructura secundaria

In [7]:
rmsf = html.Div([
    html.H4("Introducción"),
    html.P("La fluctuación cuadrática media (RMSF) representa una forma de cuantificar la flexibilidad de una molécula o una región de esta. A diferencia del RMSD que representa el desplazamiento promedio de la estructura en un instante de la simulación, el RMSF denota la fluctuación o vibración de un átomo o conjunto de átomos, con relación a una estructura de referencia"),
    html.P([html.Span("En el caso del cálculo de RMSF, para una molécula a lo largo de una simulación, se cuenta con un conjunto de estructuras que representan la evolución temporal de la molécula. La estructura de referencia usada puede ser la "),
            html.B("estructura media"),
            html.Span("que es aquella en donde cada átomo se encuentra en la posición promedio del conjunto de posiciones, o la "),
            html.B("estructura representativa"),
            html.Span("que es la que tiene más similitudes con cada estructura del conjunto, el grado de similitud se realiza mediante el computo de RMSD por pares para cada una de las integrantes del conjunto")]),
    html.P("La operación utilizada para obtener el RMSF de un solo átomo i, es la siguiente:"),
    dbc.Row([dcc.Markdown('$RMSF_{i} =\\sqrt{\\frac{1}{T} \\sum_{j=i}^{T}(\\vec{r_{j,i}}-\\frac{1}{T}\\sum_{k=i}^{T}\\vec{r_{k,l}})^2}$',mathjax=True)],justify="center",),
    html.P("Que representa la distancia promedio entre el átomo i de la estructura T y el átomo i de la estructura media/representativa, en donde se asume que las coordenadas r fueron rotadas y trasladadas para minimizar el RMSD"),

    dbc.Row([
        html.Span("Observemos como se ve cada estructura ",style={'display':'block'})]),
    html.Br(),
    html.H6("Númeración para las subunidades proteicas"),
    html.P("Puedes escoger ver los analisis que involucren información por residuo  Con 1)Numeración salida de la simulación de AMBER lo cual da facilidad para ubicar los residuos, 2)Númeracion corregida para cada subunidad proteica y mostrando numeración basada en AB para el sustrato o 3)Numeración corregida para cada subunidad proteica y numeración basada en C83 como sustrato"),
    html.Br(),
    dbc.Row([
    dbc.Col(html.Span("Escoge el formato en cual quieres numerar",style={"color":"#052080"})),
        dbc.Col(dcc.Dropdown(
            id="format-dw-pot",
            options=[{'label': 'Simulación AMBER', 'value': 'amber'},
                     {'label': 'Por proteina (Num AB)', 'value': 'ab'},
                     {'label': 'Por proteina (Num C83)', 'value': 'c83'}],
            value='ab',style={'width':"75%"}))],
        justify="center", align="center"),
    html.Br(),
    html.Div(id="plots-rmsf"),
])

ss = html.Div([
    html.H4("Analisis de estructura secundaria"),
    html.P("Veamos como se comporta la estructura de cada uno de los residuos de aminoácidos, los computos fueron relidazdos mediante CPPTRAJ con la función secstruct"),
    dbc.Row([
        html.Span("Observemos como se ve la estructura secundaria del sistema:   ",style={'display':'block'}),
        dcc.Dropdown(
                id="ss-sub",
                options=[{'label': 'Estructura Nativa', 'value': 'nat'},
                         {'label': 'Estructura Nativa3', 'value': 'gly'},
                         {'label': 'Complejo Enzima-Sustrato', 'value': 'sus'},
                         {'label': 'Complejo Enzima-Sustrato-Modulador', 'value': 'cpx'}],
                value='nat',style={'width':"70%",'display':'block'})]),
    html.Br(),
    html.H6("Númeración para las subunidades proteicas"),
    html.P("Puedes escoger ver los analisis que involucren información por residuo  Con 1)Numeración salida de la simulación de AMBER lo cual da facilidad para ubicar los residuos, 2)Númeracion corregida para cada subunidad proteica y mostrando numeración basada en AB para el sustrato o 3)Numeración corregida para cada subunidad proteica y numeración basada en C83 como sustrato"),
    html.Br(),
    dbc.Row([
    dbc.Col(html.Span("Escoge el formato en cual quieres numerar",style={"color":"#052080"})),
        dbc.Col(dcc.Dropdown(
            id="format-dw-pot",
            options=[{'label': 'Simulación AMBER', 'value': 'amber'},
                     {'label': 'Por proteina (Num AB)', 'value': 'ab'},
                     {'label': 'Por proteina (Num C83)', 'value': 'c83'}],
            value='ab',style={'width':"75%"}))],
        justify="center", align="center"),
    html.Br(),
    html.Div(id="ss-div"),
])

## Helanal y Contactos membranales

In [8]:
tilt =  html.Div([
    html.H4("Análisis de las hélices"),
    html.Br(),
    html.P("HELANAL cuantifica la geometría de las helices en proteínas en función de sus átomos Cα. Puede determinar características estructurales locales como la torsión local y la elevación, el ángulo torsional virtual, los orígenes de la hélice local y los ángulos de flexión entre ejes de hélice locales sucesivos."),
    html.Br(),
    html.P("En este apartado se realizan analisis de cada una de las 19 helices membranales de γ-secretasa"),
    html.Br(),
    dbc.Col([
        html.Span("Selecciona el analisis de relacionado a las hélices que te gustaria ver :  ",style={'display':'block'}),]),
    dbc.Col([
        dcc.Dropdown(
                id="helanalanal",
                options=[{'label': 'Tilt/Inclinacion', 'value': 'tilt'},
                        {'label': 'Ángulos entre helices', 'value': '2he'},
                        {'label': 'Kink/torcedura', 'value': 'kinky'}],
                value='tilt',style={'width':"70%",'display':'block'})]),
        
    html.Div(id="analplot")
                 ])

contete = html.Div([
    html.H4("Contactos Membranales"),
    html.Br(),
    html.P("Se realizaron todos los contactos de los residuos membranales de GS contra sigo mismos. Los contactos se presentan en mapas de calor en donde el valor de z representa el contacto normalizado por número de contactos posibles y por tiempo. Se considera contacto aquello que este a 5 Å de distancia."),
    html.Br(),
    html.P("Selecciona por favor que sistemas quieres comparar."),
    html.Br(),
    dbc.Row([       
        dcc.Dropdown(
            id="contact1",
            options=[{'label': 'Estructura Apo', 'value': 'Apo'},
                     {'label': 'Estructura Apo3', 'value': 'Gly'},
                     {'label': 'Estructura Apo l166p', 'value': 'Mut'},
                     {'label': 'Complejo Enzima-Sustrato', 'value': 'Sus'},
                     {'label': 'Complejo Enzima-Sustrato-Modulador', 'value': 'Cpx'}],
            value='Apo',style={'width':"45%",'display':'block'}),

        dcc.Dropdown(
            id="contact2",
            options=[{'label': 'Estructura Apo', 'value': 'Apo'},
                     {'label': 'Estructura Apo3', 'value': 'Gly'},
                     {'label': 'Estructura Apo l166', 'value': 'Mut'},
                     {'label': 'Complejo Enzima-Sustrato', 'value': 'Sus'},
                     {'label': 'Complejo Enzima-Sustrato-Modulador', 'value': 'Cpx'}],
            value='Mut',style={'width':"45%",'display':'block'})]),
     html.Div(id="plotcontac"),
    html.Br(),
    html.Br(),
    html.P("Por favor selecciona una helice, en esta parte vamos a ver como cambian los contactos de una helice respecto a todas las demás."),
    html.Br(),
    dcc.Dropdown(
                id="helicec",
                options=[{'label': col, 'value': col} for col in helix_df["helice1"].unique()],
                value='TM6_PS1',style={'width':"80%",'display':'block'}),
    html.Div(id="helicecplot"),
])

## Ubicaciones y PCA

In [9]:
ubis =   html.Div([html.H4("Ubicación de regiones especificas"),
                   html.P("En esta parte vamos a comparar si hay cambios en ciertas partes de GS"),
                   html.P("Escoge que region quieres analizar"),
                   html.Br(),
                   dcc.Dropdown(
                        id="u",
                        options=[{'label': '6CT', 'value': '6ct'}],
                        value='6ct',style={'width':"45%",'display':'block'}),
                   html.Div(id="ubiplis")
                  ])

pca = html.Div([html.H4("PCA"),
                html.Br(),
                html.P("Comparemos los componentes principales de las partes membranales. Selecciona que sistemas quieres comparar"),
                dbc.Row([
                         
                dcc.Dropdown(
                    id="pcas1",
                    options=[{'label': 'Estructura Apo', 'value': 'Apo'},
                             {'label': 'Estructura Apo3', 'value': 'Gly'},
                             {'label': 'Estructura Apo l166p <500', 'value': 'Mut1'},
                             {'label': 'Estructura Apo l166p >500', 'value': 'Mut2'},
                             {'label': 'Complejo Enzima-Sustrato', 'value': 'Sus'},
                             {'label': 'Complejo Enzima-Sustrato-Modulador', 'value': 'Cpx'}],
                    value='Apo',style={'width':"45%",'display':'block'}),
                         
                dcc.Dropdown(
                    id="pcas2",
                    options=[{'label': 'Estructura Apo', 'value': 'Apo'},
                             {'label': 'Estructura Apo3', 'value': 'Gly'},
                             {'label': 'Estructura Apo l166p <500', 'value': 'Mut1'},
                             {'label': 'Estructura Apo l166p >500', 'value': 'Mut2'},
                             {'label': 'Complejo Enzima-Sustrato', 'value': 'Sus'},
                             {'label': 'Complejo Enzima-Sustrato-Modulador', 'value': 'Cpx'}],
                    value='Mut1',style={'width':"45%",'display':'block'})]),
                html.Div(id="plotpca")
               ])

## Cross C. y Solvente

In [10]:
cruzado =   html.Div([html.H4("Cross Correlation")])


ws  = pd.read_csv("assets_viejos/aposhell_wat.dat",delimiter="\s+")
ws  = ws.set_index("#Frame")

ws2  = pd.read_csv("assets_viejos/mutshell_wat.dat",delimiter="\s+")
ws2  = ws2.set_index("#Frame")

fig = go.Figure()
fig.add_traces(go.Box(x = ["Primera esfera 3.4 Å<br> Apo"]*ws.shape[0], y=ws.iloc[:,0],showlegend=False,marker_color ="forestgreen",boxmean=True))
fig.add_traces(go.Box(x = ["Primera esfera 3.4 Å<br> Mut"]*ws2.shape[0], y=ws2.iloc[:,0],showlegend=False,marker_color ="firebrick",boxmean=True))
fig.add_traces(go.Box(x = ["Segunda esfera 5.0 Å<br> Apo"]*ws.shape[0], y=ws.iloc[:,1],showlegend=False,marker_color ="forestgreen",boxmean=True))
fig.add_traces(go.Box(x = ["Segunda esfera 5.0 Å<br> Mut"]*ws2.shape[0], y=ws2.iloc[:,1],showlegend=False,marker_color ="firebrick",boxmean=True))
fig.update_layout(title="Accesibilidad al solvente dentro del sitio catalitico",title_x=0.5)
fig.update_yaxes(title="Número de aguas")  


rdf = pd.read_csv("assets_viejos/apordf_wat.dat", sep="\s+")
rdf1  = pd.read_csv("assets_viejos/mutrdf_wat.dat", sep="\s+")
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=rdf.iloc[:,0], y=rdf.iloc[:,1],
                    mode='lines',marker_color="forestgreen",
                    name='Apo'))
fig1.add_trace(go.Scatter(x=rdf1.iloc[:,0], y=rdf1.iloc[:,1],
                    mode='lines',marker_color="firebrick",
                    name='Mut'))
fig1.update_layout(title="RDF Aspárticos catilícos y Agua",title_x=0.5)
fig1.update_yaxes(title="RDF") 
fig1.update_xaxes(title="r (Å)")
aguas = html.Div([
    html.H4("Accesibilidad al solvente dentro del sitio catalitico"),
    html.Br(),
    html.P("El agua desempeña un papel esencial en el mecanismo de acción de la gamma secretasa. Cuando el residuo de ácido aspártico se encuentra desprotonado, polariza una molécula de agua, lo que da inicio a la catálisis enzimática al realizar un ataque nucleofílico sobre el carbonilo del enlace que será escindido. Este proceso forma un intermediario tetraédrico que se estabiliza cuando el oxígeno del carbonilo acepta el hidrógeno ácido proveniente del ácido aspártico catalítico, generando así un intermediario gem-diol. Posteriormente, ambos residuos aspárticos vuelven a su estado de protonación inicial, lo cual resulta en la escisión del sustrato peptídico y la liberación de productos proteolíticos."),
    html.Br(),
    html.P("La presencia de agua en este mecanismo es de gran importancia. Si el agua estuviera ausente, se espera que desempeñe un papel inactivador o inhibitorio en la actividad enzimática."),
    dcc.Graph(figure=fig,config= {'toImageButtonOptions': {'format': 'png','filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1}}),
    html.Br(),
    dcc.Graph(figure=fig1,config= {'toImageButtonOptions': {'format': 'svg','filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1}}),
])

## Asparticos y lipidos

In [11]:
asp_cat = html.Div([
    html.H4("Analisis asparticos cataliticos"),
    html.P("GS es una aspartil proteasa conformada por 4 subunidades. Las cuales son faringe anterior defectuosa (APH1), presenilina (PS), potenciador de presenilina 2 (PEN2) y nicastrina (NCST), a su vez nicastrina se puede subdividir en lóbulo mayor y lóbulo menor"),
    html.P("PSEN es la subunidad catalítica de GS, se compone de 9 hélices transmembranales (TM). Los aspárticos catalíticos se encuentran en las hélices TM6 y TM7. Debido a una endoproteólisis necesaria para la maduración del complejo, PSEN está divido en sus dominios N y C terminales, los cuales se componen de los segmentos TM1-TM6 y TM7-TM9 respectivamente."),
    html.P("Al igual que otras aspartil proteasas, GS hidroliza a los sustratos mediante un mecanismo ácido-base que involucra los dos residuos de aspárticos y una molécula de agua. De manera general lo que ocurre es un ataque nucleofílico favorecido por la molécula de agua que polariza a un carboxilato. Se ha determinado mediante métodos de dinámica molecular que el aspártico catalítico 385 es aquel que se protona de manera preferente a pH fisiológico y en presencia del sustrato"),
    dbc.Row([
        html.Span("Grafiquemos la distancia de los asparticos cataliticos como :  ",style={'display':'block'}),
        dcc.Dropdown(
                id="dtplot",
                options=[{'label': 'Histograma', 'value': 'hist'},
                         {'label': 'Evolución temporal', 'value': 'tiempo'},
                         {'label': 'Histograma 2D', 'value': 'hist2D'}],
                value='hist',style={'width':"70%",'display':'block'})]),
        
    html.Div(id="dplot")
])

lipyphilic = html.Div([
    html.H4("Analisis de integridad de la membrana"),
    html.H1("Ingresa informacion de Parametro de orden y Perfil de densidad"),
    dbc.Row([
        html.Span("Selecciona el analisis de membrana que te gustaria ver :  ",style={'display':'block'}),
        dcc.Dropdown(
                id="lipy",
                options=[{'label': 'Parametro de orden', 'value': 'order'},
                         {'label': 'Perfil de densidad', 'value': 'mass'}],
                value='order',style={'width':"70%",'display':'block'})]),
        
    html.Div(id="liplot")
])

## E2012 y  ligandos

In [12]:
e2012 = html.Div([
    html.H4("Ubicacion del modulador"),
    html.Span("E2012 es un modulador de gamma secretasa tipo fenil imidazol que ha sido desarrollado como un posible tratamiento para la enfermedad de Alzheimer. Se ha demostrado que E2012 reduce la producción de amiloide beta, una proteína que se acumula en el cerebro de pacientes con Alzheimer, en estudios preclínicos en ratones transgénicos. Los ensayos clínicos en humanos también han demostrado una reducción en los niveles de amiloide beta en pacientes con enfermedad de Alzheimer tratados con E2012, aunque algunos efectos secundarios adversos, como náuseas y diarrea, se han informado en algunos pacientes. Aunque se necesita más investigación para determinar la seguridad y eficacia a largo plazo de E2012, los resultados sugieren que puede ser una opción de tratamiento prometedora para la enfermedad de Alzheimer."),
    dbc.Row([
    dbc.Col([
        html.Span("Selecciona el analisis de relacionado a e2012 que te gustaria ver :  ",style={'display':'block'}),
        dcc.Dropdown(
                id="e2012anal",
                options=[{'label': 'Ubicacion Heat-Map', 'value': 'HM'},
                         {'label': 'Analisis de contactos', 'value': 'CT'},
                        {'label': 'Puentes de Hidrógeno', 'value': 'HB'}],
                value='CT',style={'width':"70%",'display':'block'})]),
    dbc.Col([
        html.Span("Selecciona el cutoff para el analisis de contactos :  ",style={'display':'block'}),
        dcc.Dropdown(
                id="e2012contactos",
                options=[{'label': '3 Angstrom', 'value': '3A'},
                         {'label': '4 Angstrom', 'value': '4A'},
                        {'label': '5 Angstrom', 'value': '5A'},
                        {'label': '6 Angstrom', 'value': '6A'},
                        {'label': '7 Angstrom', 'value': '7A'}],
                value='5A',style={'width':"70%",'display':'block'}),
        
    ])]),
        
    html.Div(id="e2012plot")
])

In [13]:
ligandos = html.Div([
    html.H4("Moduladores"),
    html.Span("Los moduladores de gamma secretasa tipo fenil imidazol son compuestos químicos que regulan selectivamente la producción de amiloide beta, una proteína que se acumula en el cerebro de pacientes con enfermedad de Alzheimer. A diferencia de los inhibidores de gamma secretasa tradicionales, los moduladores de gamma secretasa tipo fenil imidazol reducen los efectos secundarios no deseados de la inhibición completa de la enzima. Aunque algunos estudios preclínicos han demostrado su efectividad, los ensayos clínicos en humanos han mostrado resultados mixtos y se necesita más investigación para determinar su seguridad y eficacia a largo plazo."),
    html.Br(),
    html.Br(),
    html.H4("Ligbuilderv3"),
    html.Span("LigbuilderV3 es un programa de modelado molecular utilizado en química computacional y diseño de fármacos. Su función principal es generar modelos 3D de moléculas pequeñas y optimizar su estructura mediante el ajuste de parámetros como la geometría de enlace, la energía de enlace y las interacciones intermoleculares."),
    html.Br(),
    html.Br(),
    html.H4("PERMM servidor"),
    html.Span("El método PerMM calcula la afinidad de unión a la membrana, los perfiles de energía a lo largo de la bicapa normal y los coeficientes de permeabilidad de diversas moléculas a través de diferentes membranas. Los coeficientes de permeabilidad calculados reproducen datos experimentales para diversos sistemas de membrana, como BLM (R2=0,88, RMSE=1,15 unidades logarítmicas), PAMPA (R2=0,75, RMSE=1,59 unidades logarítmicas), BBB (R2=0,69, RMSE=0,87 unidades logarítmicas). ) y células Caco-2/MDCK (R2=0,52, RMSE=0,89 unidades logarítmicas)."),
    html.Br(),
    html.H4("Recomendaciones para observar"),
    html.P("Permeabilidad BBB -4.35"),
    html.P("Eficiencia de ligando, Indice de lipofilia, Energia docking, LLE (IC50-logp)"),
    html.P("Espacio Quimico"),
    html.Br(),
    html.Br(),
    
    dbc.Row([
    dbc.Col([
        html.Span("Selecciona el analisis de relacionado a los ligandos que te gustaria ver :  ",style={'display':'block'}),]),
    dbc.Col([
        dcc.Dropdown(
                id="liganal",
                options=[{'label': 'Analisis del Servidor (Umbrella)', 'value': 'Umb'},
                        {'label': 'Dos variables', 'value': '2var'},
                        {'label': 'Relacion IC50 ', 'value': 'IC50'}],
                value='2var',style={'width':"70%",'display':'block'})])]),
    html.Br(),
    html.P("En el caso de que quieras graficar dos variables por favor por favor seleccionalas aca y selecciona el tipo de grafico que quieres que despliegue."),
    html.Br(),
    dbc.Row([
    dbc.Col([
        html.P("Variable 1:"),
        dcc.Dropdown(
                id="var1",
                options=[{'label': col, 'value': col} for col in todo.columns],
                value='lig_gr',style={'width':"80%",'display':'block'})]),
    dbc.Col([
        html.P("Variable 2:"),
        dcc.Dropdown(
                id="var2",
                options=[{'label': col, 'value': col} for col in todo.columns],
                value='vina',style={'width':"80%",'display':'block'})]),
    dbc.Col([
        html.P("Colorear por:"),
        dcc.Dropdown(
                id="color",
                options=[{'label': col, 'value': col} for col in todo.columns],
                value='lig_gr',style={'width':"80%",'display':'block'})]),
    dbc.Col([
        html.P("Tipo de gráfico:"),
        dcc.Dropdown(
                id="graph",
                options=[{'label': 'Scatter', 'value': 'scatter'},
                        {'label': 'Violin ', 'value': 'violin'}],
                value='violin',style={'width':"80%",'display':'block'})])
    
    ]),

    html.Div(id="ligplot")
    
])

# Inicio de la app

In [14]:
app = dash.Dash(__name__, external_stylesheets=[dbc.themes.SIMPLEX],
               meta_tags=[{'name': 'viewport',
                            'content': 'width=device-width, initial-scale=1.0'}])   
app.layout = html.Div([
    dcc.Location(id="url"),
    sidebar,
    content,
    dcc.Store(id='side_click'),
    dcc.Store(id="num-format"),
    dcc.Store(id="img-format"),
])

In [15]:
@app.callback(
    [
        Output("sidebar", "style"),
        Output("page-content", "style"),
        Output("side_click", "data"),
    ],

    [Input("btn_sidebar", "n_clicks")],
    [
        State("side_click", "data"),
    ]
)
def toggle_sidebar(n, nclick):
    if n:
        if nclick == "SHOW":
            sidebar_style = SIDEBAR_HIDEN
            content_style = CONTENT_STYLE1
            cur_nclick = "HIDDEN"
        else:
            sidebar_style = SIDEBAR_STYLE
            content_style = CONTENT_STYLE
            cur_nclick = "SHOW"
    else:
        sidebar_style = SIDEBAR_STYLE
        content_style = CONTENT_STYLE
        cur_nclick = 'SHOW'

    return sidebar_style, content_style, cur_nclick

# Funciones
##  Barside

In [16]:
#  Aca definimos las funciones o callback con el orden:
# * URL sidebar
# * Store img format
# * Volcano MA plot y el histograma
@app.callback(
    Output("page-content", "children"),
    [Input("url", "pathname")]
)


def render_page_content(pathname):
    if pathname == "/":
        return [encabezado,html.Hr(),Intro]
    elif pathname == "/rmsd":
        return [encabezado,p_container]
    elif pathname == "/rmsf":
        return [encabezado, html.Hr(), rmsf]
    elif pathname == "/sec-estruc":
        return [encabezado, html.Hr(), ss]
    elif pathname == "/helanal":
        return [encabezado, html.Hr(), tilt]
    elif pathname == "/contact":
        return [encabezado, html.Hr(), contete]
    elif pathname == "/ubicacion":
        return [encabezado, html.Hr(), ubis]
    elif pathname == "/pca":
        return [encabezado, html.Hr(), pca]
    elif pathname == "/cross":
        return [encabezado, html.Hr(), cruzado]
    elif pathname == "/h2o":
        return [encabezado, html.Hr(), aguas]
    elif pathname == "/asp-cat":
        return [encabezado, html.Hr(), asp_cat]
    elif pathname == "/lipyphilic":
        return [encabezado, html.Hr(), lipyphilic]
    elif pathname == "/e2012":
        return [encabezado, html.Hr(), e2012]
    elif pathname == "/lig-imi":
        return [encabezado, html.Hr(), ligandos]
    else:
        return html.Div([
            html.H1("404: Not found", className="bg-danger text-white"),
            html.H1(f"The pathname {pathname} was not recognised...") ])
@app.callback(
    Output(component_id="img-format", component_property='data'),
    Input(component_id="format-dw-img", component_property="value"),
)
def download_img_format(f):
    svg = {'toImageButtonOptions': {'format': 'svg','filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1},
          'displaylogo': False}
    png = {'toImageButtonOptions': {'format': 'png', 'filename': 'custom_image','height': 500,
                                   'width': 700,'scale': 1 },
          'displaylogo': False}

    jpeg = {'toImageButtonOptions': {'format': 'jpeg', 'filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1},
           'displaylogo': False}

    webp = {'toImageButtonOptions': {'format': 'webp','filename': 'custom_image','height': 500,
                                    'width': 700,'scale': 1},
           'displaylogo': False}
    if  f=="svg":
        return svg
    elif f=="png":
        return png
    elif f=="jpeg":
        return jpeg
    elif f=="webp":
        return webp

## RMSD

In [17]:
#********************************R M S D   P O R  E S T R U C T U R A*****************************************************   
@app.callback(
    Output("plots-rmsd", "children"),
    Input(component_id='tipormsd', component_property='value'),         #Estructura deseada
    Input(component_id='estruc', component_property='value'),         #Estructura deseada
    Input(component_id='sub1', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
)

def update_rmsd(tipo,est,sub1,f): 
    if tipo == "sis":
        if est=="nat":
            time = nat_rmsd[0][:,0]/100 
            n = go.Figure()
            n.add_traces(go.Scatter(x=time,y=nat_rmsd[0][:,1],marker_color="indianred"))
            n.update_layout(title="RMSD NCT",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=time,y=nat_rmsd[1][:,1],marker_color="goldenrod"))
            s.update_layout(title="RMSD PS1",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")

            a = go.Figure()
            a.add_traces(go.Scatter(x=time,y=nat_rmsd[2][:,1],marker_color="steelblue"))
            a.update_layout(title="RMSD APH1 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=time,y=nat_rmsd[3][:,1],marker_color="olivedrab"))
            p.update_layout(title="RMSD PEN2 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")
            fig = go.Figure()
            fig.add_traces(go.Scatter(x=time,y=nat_rmsd[4][:,1],marker_color="black"))
            fig.update_layout(title="RMSD γ-secretasa Nativa",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            fig.update_xaxes(title="Tiempo (ns)")
            fig.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=s,config=f)]),

                dbc.Col([
                    dcc.Graph(figure=a,config=f),
                    dcc.Graph(figure=p,config=f)]),

                dbc.Col([
                    dcc.Graph(figure=fig,config=f)]),
                ])])
        
        if est=="gly":
            time = nat3_rmsd[0][:,0]/100
            n = go.Figure()
            n.add_traces(go.Scatter(x=time,y=nat3_rmsd[0][:,1],marker_color="indianred"))
            n.update_layout(title="RMSD NCT",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=time,y=nat3_rmsd[1][:,1],marker_color="goldenrod"))
            s.update_layout(title="RMSD PS1",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")

            a = go.Figure()
            a.add_traces(go.Scatter(x=time,y=nat3_rmsd[2][:,1],marker_color="steelblue"))
            a.update_layout(title="RMSD APH1 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=time,y=nat3_rmsd[3][:,1],marker_color="olivedrab"))
            p.update_layout(title="RMSD PEN2 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            fig = go.Figure()
            fig.add_traces(go.Scatter(x=time,y=nat3_rmsd[4][:,1],marker_color="black"))
            fig.update_layout(title="RMSD γ-secretasa Nativa3",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            fig.update_xaxes(title="Tiempo (ns)")
            fig.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=s,config=f)]),

                dbc.Col([
                    dcc.Graph(figure=a,config=f),
                    dcc.Graph(figure=p,config=f)]),

                dbc.Col([
                    dcc.Graph(figure=fig,config=f)]),
                ])])
        if est=="sus":
            time = sus_rmsd[0][:,0]/100 

            n = go.Figure()
            n.add_traces(go.Scatter(x=time,y=sus_rmsd[0][:,1],marker_color="indianred"))
            n.update_layout(title="RMSD NCT",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=time,y=sus_rmsd[1][:,1],marker_color="goldenrod"))
            s.update_layout(title="RMSD PS1",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")

            a = go.Figure()
            a.add_traces(go.Scatter(x=time,y=sus_rmsd[2][:,1],marker_color="steelblue"))
            a.update_layout(title="RMSD APH1 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=time,y=sus_rmsd[3][:,1],marker_color="olivedrab"))
            p.update_layout(title="RMSD PEN2 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            c = go.Figure()
            c.add_traces(go.Scatter(x=time,y=sus_rmsd[4][:,1],marker_color="mediumvioletred"))
            c.update_layout(title="RMSD C83 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            c.update_xaxes(title="Tiempo (ns)")
            c.update_yaxes(title="RMSD (Å)")

            fig = go.Figure()
            fig.add_traces(go.Scatter(x=time,y=sus_rmsd[5][:,1],marker_color="black"))
            fig.update_layout(title="RMSD Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            fig.update_xaxes(title="Tiempo (ns)")
            fig.update_yaxes(title="RMSD (Å)")
            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=s,config=f),
                        dcc.Graph(figure=c,config=f)]),

                    dbc.Col([
                        dcc.Graph(figure=a,config=f),
                        dcc.Graph(figure=p,config=f),
                        dcc.Graph(figure=fig,config=f)]),
                ])])
        if est=="cpx":
            time = cpx_rmsd[0][:,0]/100 

            n = go.Figure()
            n.add_traces(go.Scatter(x=time,y=cpx_rmsd[0][:,1],marker_color="indianred"))
            n.update_layout(title="RMSD NCT",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=time,y=cpx_rmsd[1][:,1],marker_color="goldenrod"))
            s.update_layout(title="RMSD PS1",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")

            a = go.Figure()
            a.add_traces(go.Scatter(x=time,y=cpx_rmsd[2][:,1],marker_color="steelblue"))
            a.update_layout(title="RMSD APH1 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=time,y=cpx_rmsd[3][:,1],marker_color="olivedrab"))
            p.update_layout(title="RMSD PEN2 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            c = go.Figure()
            c.add_traces(go.Scatter(x=time,y=cpx_rmsd[4][:,1],marker_color="mediumvioletred"))
            c.update_layout(title="RMSD C83 ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            c.update_xaxes(title="Tiempo (ns)")
            c.update_yaxes(title="RMSD (Å)")

            fig = go.Figure()
            fig.add_traces(go.Scatter(x=time,y=cpx_rmsd[5][:,1],marker_color="black"))
            fig.update_layout(title="RMSD Complejo-Enzima-Sustrato-Modulador",title_x=0.5,yaxis_range=[0.7,5])
            fig.update_xaxes(title="Tiempo (ns)")
            fig.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=s,config=f),
                        dcc.Graph(figure=c,config=f)]),

                    dbc.Col([
                        dcc.Graph(figure=a,config=f),
                        dcc.Graph(figure=p,config=f),
                        dcc.Graph(figure=fig,config=f)]),
                ])])
        
###########################################################################################################################################################
    elif tipo =="comp":   
        if sub1=="ncst":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[0][:,0]/100 ,
                                    y=nat_rmsd[0][:,1],marker_color="indianred"))
            n.update_layout(title="RMSD NCT Estructura Nativa",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=nat3_rmsd[0][:,0]/100 ,
                                    y=nat3_rmsd[0][:,1],marker_color="indianred"))
            s.update_layout(title="RMSD NCT Estructura Nativa3",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")
            
            a = go.Figure()
            a.add_traces(go.Scatter(x=cpx_rmsd[0][:,0]/100 ,
                                    y=cpx_rmsd[0][:,1],marker_color="indianred"))
            a.update_layout(title="RMSD NCT Complejo Enzima-Sustrato-Modulador",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=sus_rmsd[0][:,0]/100 ,
                                    y=sus_rmsd[0][:,1],marker_color="indianred"))
            p.update_layout(title="RMSD NCT Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=p,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=s,config=f),
                    dcc.Graph(figure=a,config=f)],width={"size":6,"offset":0}),
                ])])
            
        if sub1=="psn1":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[1][:,0]/100 ,
                                    y=nat_rmsd[1][:,1],marker_color="goldenrod"))
            n.update_layout(title="RMSD PS1 Estructura Nativa",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=nat3_rmsd[1][:,0]/100 ,
                                    y=nat3_rmsd[1][:,1],marker_color="goldenrod"))
            s.update_layout(title="RMSD PS1 Estructura Nativa-Glicosilada",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")

            a = go.Figure()
            a.add_traces(go.Scatter(x=cpx_rmsd[1][:,0]/100 ,
                                    y=cpx_rmsd[1][:,1],marker_color="goldenrod"))
            a.update_layout(title="RMSD PS1 Complejo Enzima-Sustrato-Modulador",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=sus_rmsd[1][:,0]/100 ,
                                    y=sus_rmsd[1][:,1],marker_color="goldenrod"))
            p.update_layout(title="RMSD PS1 Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=p,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=s,config=f),
                    dcc.Graph(figure=a,config=f)],width={"size":6,"offset":0}),
                ])])
        if sub1=="aph1":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[2][:,0]/100 ,
                                    y=nat_rmsd[2][:,1],marker_color="steelblue"))
            n.update_layout(title="RMSD APH1 Estructura Nativa",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=nat3_rmsd[2][:,0]/100 ,
                                    y=nat3_rmsd[2][:,1],marker_color="steelblue"))
            s.update_layout(title="RMSD APH1 Estructura Nativa-Glicosilada",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")
            
            a = go.Figure()
            a.add_traces(go.Scatter(x=cpx_rmsd[2][:,0]/100 ,
                                    y=cpx_rmsd[2][:,1],marker_color="steelblue"))
            a.update_layout(title="RMSD APH1 Complejo Enzima-Sustrato-Modulador",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=sus_rmsd[2][:,0]/100 ,
                                    y=sus_rmsd[2][:,1],marker_color="steelblue"))
            p.update_layout(title="RMSD APH1 Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=p,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=s,config=f),
                    dcc.Graph(figure=a,config=f)],width={"size":6,"offset":0}),
                ])])
        if sub1=="pen2":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[3][:,0]/100 ,
                                    y=nat_rmsd[3][:,1],marker_color="olivedrab"))
            n.update_layout(title="RMSD PEN2 Estructura Nativa",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=nat3_rmsd[3][:,0]/100 ,
                                    y=nat3_rmsd[3][:,1],marker_color="olivedrab"))
            s.update_layout(title="RMSD PEN2 Estructura Nativa-Glicosilada",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")
        
            a = go.Figure()
            a.add_traces(go.Scatter(x=cpx_rmsd[3][:,0]/100 ,
                                    y=cpx_rmsd[3][:,1],marker_color="olivedrab"))
            a.update_layout(title="RMSD PEN2 Complejo Enzima-Sustrato-Modulador ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=sus_rmsd[3][:,0]/100 ,
                                    y=sus_rmsd[3][:,1],marker_color="olivedrab"))
            p.update_layout(title="RMSD PEN2 Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=p,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=s,config=f),
                    dcc.Graph(figure=a,config=f)],width={"size":6,"offset":0}),
                ])])
        if sub1=="sus":
            a = go.Figure()
            a.add_traces(go.Scatter(x=cpx_rmsd[4][:,0]/100 ,
                                    y=cpx_rmsd[4][:,1],marker_color="mediumvioletred"))
            a.update_layout(title="RMSD Complejo Enzima-Sustrato-Modulador ",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=sus_rmsd[4][:,0]/100 ,
                                    y=sus_rmsd[4][:,1],marker_color="mediumvioletred"))
            p.update_layout(title="RMSD Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[0.7,4.4],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=p,config=f)]),

                dbc.Col([
                    dcc.Graph(figure=a,config=f)]),
                ])])
        if sub1=="all":
            n = go.Figure()
            n.add_traces(go.Scatter(x=nat_rmsd[4][:,0]/100 ,
                                    y=nat_rmsd[4][:,1],marker_color="black"))
            n.update_layout(title="RMSD Estructura Nativa",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            n.update_xaxes(title="Tiempo (ns)")
            n.update_yaxes(title="RMSD (Å)")

            s = go.Figure()
            s.add_traces(go.Scatter(x=nat3_rmsd[4][:,0]/100 ,
                                    y=nat3_rmsd[4][:,1],marker_color="black"))
            s.update_layout(title="RMSD Estructura Nativa-Glicosilada",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            s.update_xaxes(title="Tiempo (ns)")
            s.update_yaxes(title="RMSD (Å)")

            a = go.Figure()
            a.add_traces(go.Scatter(x=cpx_rmsd[5][:,0]/100 ,
                                    y=cpx_rmsd[5][:,1],marker_color="black"))
            a.update_layout(title="RMSD Complejo Enzima-Sustrato-Modulador",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            a.update_xaxes(title="Tiempo (ns)")
            a.update_yaxes(title="RMSD (Å)")

            p = go.Figure()
            p.add_traces(go.Scatter(x=sus_rmsd[5][:,0]/100 ,
                                    y=sus_rmsd[5][:,1],marker_color="black"))
            p.update_layout(title="RMSD Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[0.7,5],font=dict(family="Courier New, monospace",size=30))
            p.update_xaxes(title="Tiempo (ns)")
            p.update_yaxes(title="RMSD (Å)")

            plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=n,config=f),
                        dcc.Graph(figure=p,config=f)],width={"size":6,"offset":0}),

                dbc.Col([
                    dcc.Graph(figure=s,config=f),
                    dcc.Graph(figure=a,config=f)],width={"size":6,"offset":0}),
                ])])
    return (plot)

## RMSF

In [18]:
#********************************R M S F   P O R  S U B U NI D A D *****************************************************   
@app.callback(
    Output("plots-rmsf", "children"),
    Input(component_id="img-format", component_property='data'),
    Input(component_id="format-dw-pot", component_property="value"),
)
def update_rmsf(f,ff):
    if  ff=="amber":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Numeracion"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Numeracion"].values.astype("str")
    elif ff=="ab":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Corregida"].values.astype("str")
    elif ff=="c83":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida2"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Corregida2"].values.astype("str")
    n = go.Figure()
    n.add_traces(go.Scatter(x=aa, y=nat_rmsf[:,1],marker_color="black"))
    n.update_layout(title="RMSF Estructura Nativa",title_x=0.5,yaxis_range=[-0.5,9.5])
    n.add_vrect(x0=aa1[0], x1=aa[666], fillcolor="indianred", opacity=0.25, line_width=0)
    n.add_vrect(x0=aa1[666], x1=aa[977], fillcolor="gold", opacity=0.25, line_width=0)
    n.add_vrect(x0=aa1[977], x1=aa[1221], fillcolor="steelblue", opacity=0.25, line_width=0)
    n.add_vrect(x0=aa1[1221], x1=aa[-1], fillcolor="olivedrab", opacity=0.25, line_width=0)
    n.update_xaxes(title="Número de residuo")
    n.update_yaxes(title="RMSF (Å)")
    ##############################################################################################
    s = go.Figure()
    s.add_traces(go.Scatter(x=aa, y=gly_rmsf[:,1],marker_color="black"))
    s.update_layout(title="RMSF Estructura Nativa Glicosilada ",title_x=0.5,yaxis_range=[-0.5,9.5])
    s.add_vrect(x0=aa[0], x1=aa[666], fillcolor="indianred", opacity=0.25, line_width=0)
    s.add_vrect(x0=aa[666], x1=aa[977], fillcolor="gold", opacity=0.25, line_width=0)
    s.add_vrect(x0=aa[977], x1=aa[1221], fillcolor="steelblue", opacity=0.25, line_width=0)
    s.add_vrect(x0=aa[1221], x1=aa[-1], fillcolor="olivedrab", opacity=0.25, line_width=0)
    s.update_xaxes(title="Número de residuo")
    s.update_yaxes(title="RMSF (Å)")
    ###########################################################################################
    p = go.Figure()
    p.add_traces(go.Scatter(x=aa1, y=cpx_rmsf[:,1],marker_color="black"))
    p.update_layout(title="RMSF Complejo Enzima-Sustrato-Modulador",title_x=0.5,yaxis_range=[-0.5,9.5])
    p.add_vrect(x0=aa1[0], x1=aa1[666], fillcolor="indianred", opacity=0.25, line_width=0)
    p.add_vrect(x0=aa1[666], x1=aa1[977], fillcolor="gold", opacity=0.25, line_width=0)
    p.add_vrect(x0=aa1[977], x1=aa1[1221], fillcolor="steelblue", opacity=0.25, line_width=0)
    p.add_vrect(x0=aa1[1221], x1=aa1[1320], fillcolor="olivedrab", opacity=0.25, line_width=0)
    p.add_vrect(x0=aa1[1320], x1=aa1[-1], fillcolor="mediumvioletred", opacity=0.25, line_width=0)
    p.update_xaxes(title="Número de residuo")
    p.update_yaxes(title="RMSF (Å)")
    ################################################################################################
    a = go.Figure()
    a.add_traces(go.Scatter(x=aa1, y=sus_rmsf[:,1],marker_color="black"))
    a.update_layout(title="RMSF Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[-0.5,9.5])
    a.add_vrect(x0=aa1[0], x1=aa1[666], fillcolor="indianred", opacity=0.25, line_width=0)
    a.add_vrect(x0=aa1[666], x1=aa1[977], fillcolor="gold", opacity=0.25, line_width=0)
    a.add_vrect(x0=aa1[977], x1=aa1[1221], fillcolor="steelblue", opacity=0.25, line_width=0)
    a.add_vrect(x0=aa1[1221], x1=aa1[1320], fillcolor="olivedrab", opacity=0.25, line_width=0)
    a.add_vrect(x0=aa1[1320], x1=aa1[-1], fillcolor="mediumvioletred", opacity=0.25, line_width=0)
    a.update_xaxes(title="Número de residuo")
    a.update_yaxes(title="RMSF (Å)")
    
    fig = go.Figure()
    fig.update_layout(title="RMSF Sistemas de GS",title_x=0.5,yaxis_range=[-0.5,9.5])
    fig.add_traces(go.Scatter(x=aa, y=nat_rmsf[:,1],marker_color="green",name="Estructura Nativa"))
    fig.add_traces(go.Scatter(x=aa, y=gly_rmsf[:,1],marker_color="teal",name="Estructura Nativa-Glicosilada"))
    fig.add_traces(go.Scatter(x=aa1, y=sus_rmsf[:,1],marker_color="mediumblue",name="RMSF Complejo Enzima-Sustrato"))
    fig.add_traces(go.Scatter(x=aa1, y=cpx_rmsf[:,1],marker_color="purple",name="RMSF Complejo Enzima-Sustrato-Modulador"))
    fig.add_vrect(x0=aa[0], x1=aa[666], fillcolor="indianred", opacity=0.25, line_width=0)
    fig.add_vrect(x0=aa[666], x1=aa[977], fillcolor="gold", opacity=0.25, line_width=0)
    fig.add_vrect(x0=aa[977], x1=aa[1221], fillcolor="steelblue", opacity=0.25, line_width=0)
    fig.add_vrect(x0=aa[1221], x1=aa[1321], fillcolor="olivedrab", opacity=0.25, line_width=0)
    fig.add_vrect(x0=aa1[1321], x1=aa1[1360], fillcolor="mediumvioletred", opacity=0.25, line_width=0)
    fig.update_xaxes(title="Número de residuo")
    fig.update_yaxes(title="RMSF (Å)")
    
    plot= html.Div([
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=n,config=f),
            dcc.Graph(figure=a,config=f)]),

    dbc.Col([
        dcc.Graph(figure=s,config=f),
        dcc.Graph(figure=p,config=f)]),
    html.Br(),
    dbc.Row(dcc.Graph(figure=fig,config=f))
    ])])
    return (plot)

## Estructura secundaria

In [19]:
#******************************** E S T R U C T U R A   S E C U N D A R I A *****************************************************   
@app.callback(
    Output("ss-div", "children"),
    Input(component_id='ss-sub', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
    Input(component_id="format-dw-pot", component_property="value"),
)
def ssk(kind,f,ff):
    if  ff=="amber":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Numeracion"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Numeracion"].values.astype("str")
    elif ff=="ab":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Corregida"].values.astype("str")
    elif ff=="c83":
        aa = df[df["Sistema"]=="Estructura Nativa"]["Subunidad"].values+": "+df[df["Sistema"]=="Estructura Nativa"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Estructura Nativa"]["Corregida2"].values.astype("str")
        aa1 = df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Subunidad"].values+": "+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Aminoacido"].values+"-"+df[df["Sistema"]=="Complejo Enzima-Sustrato"]["Corregida2"].values.astype("str")
        
    if kind=="nat":
        ss_ss = pd.read_csv('assets/ssecondary_nat.dat.sum',sep="\s+")
        imagen = "Estructura secundaria_nat.svg"
        title="Estructura Secundaria" 
    #Estructura te chunk de codigo es para dejar solo la estructura mas representativa en las laminas beta
    ss_ss["Fracc" ]= ss_ss["Para"].values+ss_ss["Anti"].values+ss_ss["3-10"].values+ss_ss["Alpha"].values+ss_ss["Pi"].values+ss_ss["Turn"].values+ss_ss["Bend"].values
    mn = ss_ss[ss_ss["Fracc"]>1.5].iloc[:,1:3].values.argmin(axis=1) +1
    ind = ss_ss[ss_ss["Fracc"]>1.5].index
    for i in range(len(mn)):
        ss_ss.iloc[ind[i],mn[i]] = 0
    ss_ss["Fracc" ]= ss_ss["Para"].values+ss_ss["Anti"].values+ss_ss["3-10"].values+ss_ss["Alpha"].values+ss_ss["Pi"].values+ss_ss["Turn"].values+ss_ss["Bend"].values
    #aca acaba el chunk

    colores=["steelblue","midnightblue","maroon","firebrick","darkgoldenrod","palegoldenrod", "beige"]
    tipos =['Para',"Anti","3-10","Alpha","Pi","Turn", "Bend"]
    fig = go.Figure()
    for i in range(len(tipos)):
        fig.add_traces(go.Bar(x=amino, y=ss_ss[tipos[i]],name=tipos[i],
                             marker_color=colores[i]))
        fig.update_layout(title_x=0.5,title_text=title,barmode="stack",width=2000,height=400,)
        fig.update_xaxes(title="Residuos")
        fig.update_yaxes(title="Probabilidad")
    amino =aa
    if kind=="gly":
        ss_ss = pd.read_csv('assets/ssecondary_gly.dat.sum',sep="\s+")
        imagen = "Estructura secundaria_gly.svg"
        title="Estructura Secundaria Estructura Nativa Glicosilada"
        amino =aa
    elif kind=="mut":
        ss_ss = pd.read_csv('assets/ssecondary_mut.dat.sum',sep="\s+")
        imagen = "Estructura secundaria_mut.svg"
        title="Estructura Secundaria Estructura Nativa L166P"
        amino =aa
    elif kind =="cpx":
        ss_ss = pd.read_csv('assets/ssecondary_cpx.dat.sum',sep="\s+")
        imagen = "Estructura secundaria_cpx.svg"
        title="Estructura Secundaria Complejo Enzima-Sustrato-Modulador"
        amino =aa1
    elif kind=="sus":
        ss_ss = pd.read_csv('assets/ssecondary_sus.dat.sum',sep="\s+")
        imagen = "Estructura secundaria_sus.svg"
        title="Estructura Secundaria Complejo Enzima-Sustrato"
        amino =aa1
    #Este chunk de codigo es para dejar solo la estructura mas representativa en las laminas beta
    ss_ss["Fracc" ]= ss_ss["Para"].values+ss_ss["Anti"].values+ss_ss["3-10"].values+ss_ss["Alpha"].values+ss_ss["Pi"].values+ss_ss["Turn"].values+ss_ss["Bend"].values
    mn = ss_ss[ss_ss["Fracc"]>1.5].iloc[:,1:3].values.argmin(axis=1) +1
    ind = ss_ss[ss_ss["Fracc"]>1.5].index
    for i in range(len(mn)):
        ss_ss.iloc[ind[i],mn[i]] = 0
    ss_ss["Fracc" ]= ss_ss["Para"].values+ss_ss["Anti"].values+ss_ss["3-10"].values+ss_ss["Alpha"].values+ss_ss["Pi"].values+ss_ss["Turn"].values+ss_ss["Bend"].values
    #aca acaba el chunk
    
    colores=["steelblue","midnightblue","maroon","firebrick","darkgoldenrod","palegoldenrod", "beige"]
    tipos =['Para',"Anti","3-10","Alpha","Pi","Turn", "Bend"]
    fig = go.Figure()
    for i in range(len(tipos)):
        fig.add_traces(go.Bar(x=amino, y=ss_ss[tipos[i]],name=tipos[i],
                             marker_color=colores[i]))
    fig.update_layout(title_x=0.5,title_text=title,barmode="stack",width=2000,height=400,)
    fig.update_xaxes(title="Residuos")
    fig.update_yaxes(title="Probabilidad")
    
    plot=html.Div([dbc.Row(dcc.Graph(figure=fig,config=f)),
                   html.Br(),html.Br(),
                   dbc.Col(html.Img(src="assets/"+imagen,style={ 'width':'100%','height':'100%'},className="d-flex align-self-center"))
                  ])
    return plot

## HELANAL

In [20]:
#******************************** H E L A N A L  **********************************************************   
@app.callback(
    Output("pcaplot", "children"),
    Input(component_id='helanalanal', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
)
def dist_cat(tipo,f):  
    if tipo=="tilt": 
        plot= html.Div([
        html.Img(src="assets/Tilt_dots.png",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center"),
        html.Br(),
        html.Img(src="assets/Tilt.svg",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center"),
        ])
    elif tipo =="2he":
        plot= html.Div([
        html.Img(src="assets/helices_angulo.png",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center"),
        html.Br(),])
    elif tipo=="kinky":
            plot= html.Div([
        html.Img(src="assets/kinks.png",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center"),
        html.Br()])
    return plot
        

## Contactos membranales

In [21]:
#******************************** C O N T A C T O S  **********************************************************   
@app.callback(
    Output("plotcontac", "children"),
    Input(component_id='contact1', component_property='value'),         #Estructura deseada
    Input(component_id='contact2', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
)
def contacmat(pcas1,pcas2,f): 
    if pcas1 =="Apo":
        mat1 = apo_cont
    elif pcas1 =="Gly":
        mat1 = gly_cont
    elif pcas1 =="Mut":
        mat1 = mut_cont
    elif pcas1 =="Sus":
        mat1 = sus_cont
    elif pcas1 =="Cpx":
        mat1 = cpx_cont
    if pcas2 =="Apo":
        mat2 = apo_cont
    elif pcas2 =="Gly":
        mat2 = gly_cont
    elif pcas2 =="Mut":
        mat2 = mut_cont
    elif pcas2 =="Sus":
        mat2 = sus_cont
    elif pcas2 =="Cpx":
        mat2 = cpx_cont
    resta = mat1-mat2
    resta2 = mat2-mat1
    resta = np.where(resta <= 0, 0, resta)
    resta2 = np.where(resta2 <= 0, 0, resta2)
    
    fig = make_subplots(rows=1,cols=2,subplot_titles=["Contactos Sistema "+pcas1,"Contactos Sistema "+pcas2 ])
    fig.add_trace(go.Heatmap(
                       z=mat1,
                       x=aa[635:-1],
                       y=aa[635:-1],colorscale="Aggrnyl"),row=1,col=1)

    fig.add_trace(go.Heatmap(
                       z=mat2,
                       x=aa[635:-1],
                       y=aa[635:-1],colorscale="Aggrnyl"),row=1,col=2)

    fig.update_layout(height=1200, width=2000, title_text="Contactos membranales",title_x=0.5)
    fig.update_xaxes(tickangle=-90)
    
    fig2 = make_subplots(rows=1,cols=2,subplot_titles=["Si en "+pcas1+" no en "+pcas2, "Si en "+pcas2+" no en "+pcas1])
    fig2.add_trace(go.Heatmap(
                       z=np.triu(resta),
                       x=aa[635:-1],
                       y=aa[635:-1],colorscale="Aggrnyl"),row=1,col=1)

    fig2.add_trace(go.Heatmap(
                       z=np.triu(resta2),
                       x=aa[635:-1],
                       y=aa[635:-1],colorscale="Aggrnyl"),row=1,col=2)

    fig2.update_layout(height=1200, width=2000, title_text="Contactos membranales",title_x=0.5)
    fig2.update_xaxes(tickangle=-90)
        
    plotconta = html.Div([
        dbc.Row(dcc.Graph(figure=fig,config=f)),
        html.Br(),
        html.Br(),
        dbc.Row(dcc.Graph(figure=fig2,config=f))])
    return plotconta


#******************************** C O N T A C T O S  **********************************************************   
@app.callback(
    Output("helicecplot", "children"),
    Input(component_id='helicec', component_property='value'),
    Input(component_id="img-format", component_property='data'),
)
def helix_conana(helicec,f): 
    fig3 = px.bar(helix_df[helix_df["helice1"]==helicec], x='Contactos', y='Fracción',color="Sistema", barmode="group",
              color_discrete_sequence=['forestgreen','firebrick'],title="Contactos con "+helicec)
    
    plothelicon =html.Div([
        dbc.Row(dcc.Graph(figure=fig3,config=f))])
    return plothelicon

## Ubicacion de regiones 

In [22]:
#******************************** U B I C A C I O N   **********************************************************   
@app.callback(
    Output("ubiplis", "children"),
    Input(component_id='u', component_property='value'),
    Input(component_id="img-format", component_property='data'),
)
def ubiporfis(u,f): 
    if u=="6ct":
        fig = go.Figure()
        fig.add_traces(go.Violin(y=asp1.iloc[:,1],name="Asp Apo",meanline_visible=True,line_color="forestgreen"))
        fig.add_traces(go.Violin(y=asp.iloc[:,1],name="Asp Mut",meanline_visible=True,line_color="firebrick"))

        fig.add_traces(go.Violin(y=p311.iloc[:,1],name="P31 lower Apo",meanline_visible=True,line_color="forestgreen"))
        fig.add_traces(go.Violin(y=p31.iloc[:,1],name="P31 lower Mut",meanline_visible=True,line_color="firebrick"))

        fig.add_traces(go.Violin(y=popc1.iloc[:,1],name="POPC Apo",meanline_visible=True,line_color="forestgreen"))
        fig.add_traces(go.Violin(y=popc.iloc[:,1],name="POPC Mut",meanline_visible=True,line_color="firebrick"))

        fig.add_traces(go.Violin(y=tm21.iloc[:,1],name="Loop TM2-TM3 Apo",meanline_visible=True,line_color="forestgreen"))
        fig.add_traces(go.Violin(y=tm2.iloc[:,1],name="Loop TM2-TM3 Mut",meanline_visible=True,line_color="firebrick"))

        fig.update_xaxes(title="Punto de referencia",tickangle=-90)
        fig.update_yaxes(title="Distancia (Å)")
        fig.update_layout(title="Distancia de centros de masa <br> Helice 6A  - ", title_x=0.5)
        fig2 = go.Figure()
        fig2.add_traces(go.Violin(y=difz31,name="P31 lower Apo",meanline_visible=True,line_color="forestgreen"))
        fig2.add_traces(go.Violin(y=difz311,name="P31 lower Mut",meanline_visible=True,line_color="firebrick"))

        fig2.add_traces(go.Violin(y=difzpopc,name="POPC Apo",meanline_visible=True,line_color="forestgreen"))
        fig2.add_traces(go.Violin(y=difzpopc1,name="POPC Mut",meanline_visible=True,line_color="firebrick"))


        fig2.update_layout(title="Distancia de centros de masa <br> Helice 6A  - ",title_x=0.5)
        fig2.update_yaxes(title="Distancia (Å)")
    plotubiubi =html.Div([
        dbc.Row(dcc.Graph(figure=fig,config=f)),
         dbc.Row(dcc.Graph(figure=fig2,config=f)),
    ])
    return plotubiubi

## PCA

In [23]:
#******************************** P C A  **********************************************************   
@app.callback(
    Output("plotpca", "children"),
    Input(component_id='pcas1', component_property='value'),         #Estructura deseada
    Input(component_id='pcas2', component_property='value'),         #Estructura deseada
)
def dist_cat(pcas1,pcas2): 
    plotpca = html.Div([
        html.B("γ-Secretasa "+pcas1),
        dbc.Row([
            dbc.Col([
                html.B("PC1"),
                html.Img(src="assets_viejos/pca1"+pcas1+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]),
            dbc.Col([
                html.B("PC2"),
                html.Img(src="assets_viejos/pca2"+pcas1+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]),
            dbc.Col([
                html.B("PC3"),
                html.Img(src="assets_viejos/pca3"+pcas1+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]), 
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.Br(),
            html.B("γ-Secretasa "+pcas2),
        dbc.Row([
            dbc.Col([
                html.B("PC1"),
                html.Img(src="assets_viejos/pca1"+pcas2+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]),
            dbc.Col([
                html.B("PC2"),
                html.Img(src="assets_viejos/pca2"+pcas2+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]),
            dbc.Col([
                html.B("PC3"),
                html.Img(src="assets_viejos/pca3"+pcas2+".gif",style={'height':'130%', 'width':'95%'},className="d-flex align-self-center")]), 

    ])])])
    return plotpca

##  Cross Correlation

## Asparticos

In [24]:
#******************************** D I S T A N C I A   D E   A S P A R T I C O S*****************************************************   
@app.callback(
    Output("dplot", "children"),
    Input(component_id='dtplot', component_property='value'),         #Estructura deseada
    Input(component_id="img-format", component_property='data'),
)
def dist_cat(kind,f):  
    if kind=="hist":
        a = go.Figure(data=[go.Histogram(x=nat_dist_d[:,1], histnorm='probability',marker_color="royalblue",hovertext="Media: "+str(nat_dist_d[:,1].mean()))])
        a.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa",title_x=0.5,yaxis_range=[0,0.026],xaxis_range=[3.5,11])
        a.update_yaxes(title="Probability")
        a.update_xaxes(title="Distancia (Å)")
        b = go.Figure(data=[go.Histogram(x=nat3_dist_d[:,1], histnorm='probability',marker_color="royalblue",hovertext="Media: "+str(nat3_dist_d[:,1].mean()))])
        b.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa3",title_x=0.5,yaxis_range=[0,0.026],xaxis_range=[3.5,11])
        b.update_yaxes(title="Probability")
        b.update_xaxes(title="Distancia (Å)")
        c = go.Figure(data=[go.Histogram(x=cpx_dist_d[:,1], histnorm='probability',marker_color="royalblue",hovertext="Media: "+str(cpx_dist_d[:,1].mean()))])
        c.update_layout(title="Distancia entre los asparticos cataliticos <br> Complejo Enzima-Sustrato-Modulador",title_x=0.5,yaxis_range=[0,0.026],xaxis_range=[3.5,11])
        c.update_yaxes(title="Probability")
        c.update_xaxes(title="Distancia (Å)")
        d = go.Figure(data=[go.Histogram(x=sus_dist_d[:,1], histnorm='probability',marker_color="royalblue",hovertext="Media: "+str(sus_dist_d[:,1].mean()))])
        d.update_layout(title="Distancia entre los asparticos cataliticos <br> Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[0,0.026],xaxis_range=[3.5,11])
        d.update_yaxes(title="Probability")
        d.update_xaxes(title="Distancia (Å)")
        e = go.Figure(data=[go.Histogram(x=cpx3_dist_d[:,1], histnorm='probability',marker_color="royalblue",hovertext="Media: "+str(cpx3_dist_d[:,1].mean()))])
        e.update_layout(title="Distancia entre los asparticos cataliticos <br> Complejo Enzima-Sustrato-Modulador3",title_x=0.5,yaxis_range=[0,0.026],xaxis_range=[3.5,11])
        e.update_yaxes(title="Probability")
        e.update_xaxes(title="Distancia (Å)")
        plot= html.Div([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=a,config=f),
                    dcc.Graph(figure=d,config=f)],width={"size":4,"offset":0}),
            dbc.Col([
                dcc.Graph(figure=b,config=f),
                dcc.Graph(figure=c,config=f)],width={"size":4,"offset":0}),
            dbc.Col([dcc.Graph(figure=e,config=f)],width={"size":4,"offset":0}),
            ])])
    if kind=="tiempo":
        a = go.Figure()
        a.add_traces(go.Scatter(x=nat_dist_d[:,0],y=nat_dist_d[:,1],marker_color="royalblue"))
        a.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa",title_x=0.5,yaxis_range=[3,11])
        a.update_xaxes(title="Tiempo (ns)")
        a.update_yaxes(title="Distancia (Å)")
        b = go.Figure()
        b.add_traces(go.Scatter(x=nat3_dist_d[:,0],y=nat3_dist_d[:,1],marker_color="royalblue"))
        b.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa-Glicosilada",title_x=0.5,yaxis_range=[3,11])
        b.update_xaxes(title="Tiempo (ns)")
        b.update_yaxes(title="Distancia (Å)")
        c = go.Figure()
        c.add_traces(go.Scatter(x=cpx_dist_d[:,0],y=cpx_dist_d[:,1],marker_color="royalblue"))
        c.update_layout(title="Distancia entre los asparticos cataliticos <br> Complejo Enzima-Sustrato-Modulador",title_x=0.5,yaxis_range=[3,11])
        c.update_xaxes(title="Tiempo (ns)")
        c.update_yaxes(title="Distancia (Å)")
        d = go.Figure()
        d.add_traces(go.Scatter(x=sus_dist_d[:,0],y=sus_dist_d[:,1],marker_color="royalblue"))
        d.update_layout(title="Distancia entre los asparticos cataliticos <br> Complejo Enzima-Sustrato",title_x=0.5,yaxis_range=[3,11])
        d.update_xaxes(title="Tiempo (ns)")
        d.update_yaxes(title="Distancia (Å)")
        e = go.Figure()
        e.add_traces(go.Scatter(x=cpx3_dist_d[:,0],y=cpx3_dist_d[:,1],marker_color="royalblue"))
        e.update_layout(title="Distancia entre los asparticos cataliticos <br> Estructura Nativa-L166P",title_x=0.5,yaxis_range=[3,11])
        e.update_xaxes(title="Tiempo (ns)")
        e.update_yaxes(title="Distancia (Å)")
        plot= html.Div([
                dbc.Row([
                    dbc.Col([
                        dcc.Graph(figure=a,config=f),
                        dcc.Graph(figure=d,config=f)],width={"size":4,"offset":0}),
                dbc.Col([
                    dcc.Graph(figure=b,config=f),
                    dcc.Graph(figure=c,config=f)],width={"size":4,"offset":0}),
                dbc.Col([dcc.Graph(figure=e,config=f)],width={"size":4,"offset":0}),
                ])])
    return plot

## Lipidos

In [25]:
#********************************A N A L I S I S  D E  L I P I D O S*****************************************************   
@app.callback(
    Output("liplot", "children"),
    Input(component_id="lipy", component_property='value'),
    Input(component_id="img-format", component_property='data'),
)
def lipidos(ana,f):
    if ana =="order":
        order_nat  = np.genfromtxt("assets/order_density_nat.dat")
        order_gly  = np.genfromtxt("assets/order_density_gly.dat")
        order_mut  = np.genfromtxt("assets/order_density_mut.dat")
        order_cpx  = np.genfromtxt("assets/order_density_cpx.dat")
        order_sus  = np.genfromtxt("assets/order_density_sus.dat")
        n = go.Figure()
        n.add_traces(go.Scatter(x=[i for i in range(2,17)],y=np.mean(order_nat[:,1:6:2],axis=1),name="Palmitoil",line=dict(color="steelblue"),error_y=dict(type='data',array=np.mean(order_nat[:,2:7:2],axis=1),visible=True)))
        n.add_traces(go.Scatter(x=[i for i in range(2,19)],y=np.mean(order_nat[:,7::2],axis=1),name="Oleoil",line=dict(color="indianred"),error_y=dict(type='data',array=np.mean(order_nat[:,8::2],axis=1),visible=True)))
        n.update_layout(title="Sistema: Estructura Nativa <br> Parametro de Orden ",title_x=0.5,yaxis_range=[-0.05,0.25])
        n.update_xaxes(title="Número de carbono")
        n.update_yaxes(title="SCD")
        s = go.Figure()
        s.add_traces(go.Scatter(x=[i for i in range(2,17)],y=np.mean(order_gly[:,1:6:2],axis=1),name="Palmitoil",line=dict(color="steelblue"),error_y=dict(type='data',array=np.mean(order_gly[:,2:7:2],axis=1),visible=True)))
        s.add_traces(go.Scatter(x=[i for i in range(2,19)],y=np.mean(order_gly[:,7::2],axis=1),name="Oleoil",line=dict(color="indianred"),error_y=dict(type='data',array=np.mean(order_gly[:,8::2],axis=1),visible=True)))
        s.update_layout(title="Sistema: Estructura Nativa Glicosilada <br> Parametro de Orden ",title_x=0.5,yaxis_range=[-0.05,0.25])
        s.update_xaxes(title="Número de carbono")
        s.update_yaxes(title="SCD")
        m = go.Figure()
        m.add_traces(go.Scatter(x=[i for i in range(2,17)],y=np.mean(order_mut[:,1:6:2],axis=1),name="Palmitoil",line=dict(color="steelblue"),error_y=dict(type='data',array=np.mean(order_mut[:,2:7:2],axis=1),visible=True)))
        m.add_traces(go.Scatter(x=[i for i in range(2,19)],y=np.mean(order_mut[:,7::2],axis=1),name="Oleoil",line=dict(color="indianred"),error_y=dict(type='data',array=np.mean(order_mut[:,8::2],axis=1),visible=True)))
        m.update_layout(title="Sistema: Estructura Nativa L166P <br> Parametro de Orden ",title_x=0.5,yaxis_range=[-0.05,0.25])
        m.update_xaxes(title="Número de carbono")
        m.update_yaxes(title="SCD")
        p = go.Figure()
        p.add_traces(go.Scatter(x=[i for i in range(2,17)],y=np.mean(order_cpx[:,1:6:2],axis=1),name="Palmitoil",line=dict(color="steelblue"),error_y=dict(type='data',array=np.mean(order_cpx[:,2:7:2],axis=1),visible=True)))
        p.add_traces(go.Scatter(x=[i for i in range(2,19)],y=np.mean(order_cpx[:,7::2],axis=1),name="Oleoil",line=dict(color="indianred"),error_y=dict(type='data',array=np.mean(order_cpx[:,8::2],axis=1),visible=True)))
        p.update_layout(title="Sistema: Complejo Enzima-Sustrato-Modulador <br> Parametro de Orden ",title_x=0.5,yaxis_range=[-0.05,0.25])
        p.update_xaxes(title="Número de carbono")
        p.update_yaxes(title="SCD")
        a = go.Figure()
        a.add_traces(go.Scatter(x=[i for i in range(2,17)],y=np.mean(order_sus[:,1:6:2],axis=1),name="Palmitoil",line=dict(color="steelblue"),error_y=dict(type='data',array=np.mean(order_sus[:,2:7:2],axis=1),visible=True)))
        a.add_traces(go.Scatter(x=[i for i in range(2,19)],y=np.mean(order_sus[:,7::2],axis=1),name="Oleoil",line=dict(color="indianred"),error_y=dict(type='data',array=np.mean(order_sus[:,8::2],axis=1),visible=True)))
        a.update_layout(title="Sistema: Complejo Enzima-Sustrato <br> Parametro de Orden ",title_x=0.5,yaxis_range=[-0.05,0.25])
        a.update_xaxes(title="Número de carbono")
        a.update_yaxes(title="SCD")
    #########################################################################################################################################################################
    if ana =="mass":
        color = [None, "slategray","silver","darkgray","lightgrey","darkgray","lightgrey","royalblue","cornflowerblue","firebrick","tomato","black","gray", "purple","violet"]
        names =["#Z","PC", "sd(:PC)","OL","sd(:OL)","PA","sd(:PA)","WAT","sd(:WAT)","POPC","sdPOPC","Protein","sdProtein"]
        density_nat  = pd.read_csv("assets/nat_density.csv").values
        density_gly  = pd.read_csv("assets/gly_density.csv").values
        density_cpx  = pd.read_csv("assets/cpx_density.csv").values
        density_sus  = pd.read_csv("assets/sus_density.csv").values
        density_mut  = pd.read_csv("assets/mut_density.csv").values
        n = go.Figure()
        for i in range (1,density_nat.shape[1],2):
            n.add_traces(go.Scatter(x=density_nat[:,0],y=density_nat[:,i]+density_nat[:,i+1],showlegend=False,marker_color=color[i+1]))
            n.add_traces(go.Scatter(x=density_nat[:,0],y=density_nat[:,i]-density_nat[:,i+1],showlegend=False,marker_color=color[i+1],
                                      fill="tonexty"))
            n.add_traces(go.Scatter(x=density_nat[:,0],y=density_nat[:,i],name=names[i],
                                     marker_color=color[i]))
        n.update_layout(title="Sistema: Estructura Nativa <br> Perfil de densidad de membrana <br> Ancho membrana: "+anchomem(density_nat) +"Å",title_x=0.5)
        n.update_xaxes(title="Eje Z (Å)")
        n.update_yaxes(title="Densidad kg/m³")
        s = go.Figure()
        for i in range (1,density_gly.shape[1],2):
            s.add_traces(go.Scatter(x=density_gly[:,0],y=density_gly[:,i]+density_gly[:,i+1],showlegend=False,marker_color=color[i+1]))
            s.add_traces(go.Scatter(x=density_gly[:,0],y=density_gly[:,i]-density_gly[:,i+1],showlegend=False,marker_color=color[i+1],
                                      fill="tonexty"))
            s.add_traces(go.Scatter(x=density_gly[:,0],y=density_gly[:,i],name=names[i],
                                     marker_color=color[i]))
        s.update_layout(title="Sistema: Estructura Nativa Glicosilada <br> Perfil de densidad de membrana <br> Ancho membrana: "+anchomem(density_gly) +"Å",title_x=0.5)
        s.update_xaxes(title="Eje Z (Å)")
        s.update_yaxes(title="Densidad kg/m³")
        m = go.Figure()
        for i in range (1,density_mut.shape[1],2):
            m.add_traces(go.Scatter(x=density_mut[:,0],y=density_mut[:,i]+density_mut[:,i+1],showlegend=False,marker_color=color[i+1]))
            m.add_traces(go.Scatter(x=density_mut[:,0],y=density_mut[:,i]-density_mut[:,i+1],showlegend=False,marker_color=color[i+1],
                                      fill="tonexty"))
            m.add_traces(go.Scatter(x=density_mut[:,0],y=density_mut[:,i],name=names[i],
                                     marker_color=color[i]))
        m.update_layout(title="Sistema: Estructura Nativa L166P <br> Perfil de densidad de membrana <br> Ancho membrana: "+anchomem(density_mut) +"Å",title_x=0.5)
        m.update_xaxes(title="Eje Z (Å)")
        m.update_yaxes(title="Densidad kg/m³")
        p = go.Figure()
        for i in range (1,density_cpx.shape[1],2):
            p.add_traces(go.Scatter(x=density_cpx[:,0],y=density_cpx[:,i]+density_cpx[:,i+1],showlegend=False,marker_color=color[i+1]))
            p.add_traces(go.Scatter(x=density_cpx[:,0],y=density_cpx[:,i]-density_cpx[:,i+1],showlegend=False,marker_color=color[i+1],
                                      fill="tonexty"))
            p.add_traces(go.Scatter(x=density_cpx[:,0],y=density_cpx[:,i],name=names[i],
                                     marker_color=color[i]))
        p.update_layout(title="Sistema: Complejo Enzima-Sustrato-Modulador <br> Perfil de densidad de membrana <br> Ancho membrana: "+anchomem(density_cpx) +"Å",title_x=0.5)
        p.update_xaxes(title="Eje Z (Å)")
        p.update_yaxes(title="Densidad kg/m³")
        density_sus  = pd.read_csv("assets/nat_density.csv").values
        a = go.Figure()
        for i in range (1,density_sus.shape[1],2):
            a.add_traces(go.Scatter(x=density_sus[:,0],y=density_sus[:,i]+density_sus[:,i+1],showlegend=False,marker_color=color[i+1]))
            a.add_traces(go.Scatter(x=density_sus[:,0],y=density_sus[:,i]-density_sus[:,i+1],showlegend=False,marker_color=color[i+1],
                                      fill="tonexty"))
            a.add_traces(go.Scatter(x=density_sus[:,0],y=density_sus[:,i],name=names[i],
                                     marker_color=color[i]))
        a.update_layout(title="Sistema: Complejo Enzima-Sustrato <br> Perfil de densidad de membrana <br> Ancho membrana: "+anchomem(density_sus) +"Å",title_x=0.5)
        a.update_xaxes(title="Eje Z (Å)")
        a.update_yaxes(title="Densidad kg/m³")
    ##############################################################################################################################################################################################################
    
    plot= html.Div([
    dbc.Row([
        dbc.Col([
            dcc.Graph(figure=n,config=f),
            dcc.Graph(figure=a,config=f)],width={"size":4,"offset":0}),

    dbc.Col([
        dcc.Graph(figure=s,config=f),
        dcc.Graph(figure=p,config=f)],width={"size":4,"offset":0}),
    ]),
    dbc.Col([dcc.Graph(figure=m,config=f)],width={"size":4,"offset":0})])
    return (plot)

## E2012 

In [26]:
#******************************** A N A L I S I S  D E  M O D U L A D O R *****************************************************   
@app.callback(
    Output("e2012plot", "children"),
    Input(component_id="e2012anal", component_property='value'),
    Input(component_id="img-format", component_property='data'),
    Input(component_id="e2012contactos",component_property='value'),
)
def E2012(ana,f,CT):
    c2="indianred"
    
    if ana =="HM":
        fig2 = go.Figure()
        fig2.add_traces(go.Heatmap(z=didi["Distancias-all"].values,
                                  x=['NCST-ILE-242', 'NCST-ASN-243', 'PSEN-PHE-105', 'PSEN-TYR-106', 'PSEN-ASP-110', 'PSEN-GLY-111','PSEN-GLN-112', 'PSEN-LEU-113', 'PSEN-ILE-114', 'PSEN-TYR-115', 'PSEN-THR-116', 'PSEN-PRO-117','PSEN-LEU-173', 'PSEN-PHE-176', 'PSEN-PHE-177', 'PSEN-SER-178', 'PSEN-ILE-180', 'PSEN-TYR-181','PSEN-GLU-184', 'PSEN-ALA-231', 'PSEN-LEU-232', 'PSEN-MET-233', 'PSEN-ALA-234', 'PSEN-LEU-235','PSEN-VAL-236', 'PSEN-PHE-237', 'PSEN-LYS-239', 'PSEN-TYR-240','AB-GLY-8', 'AB-SER-9','AB-ASN-10', 'AB-LYS-11', 'AB-ILE-14', 'AB-ILE-15','AB-GLY-16', 'AB-MET-18', 'AB-VAL-19', 'AB-GLY-20', 'AB-GLY-21','AB-VAL-22']*6,
                                  y=didi["Estructura"].values,
                                  colorscale='OrRd',reversescale=True))
        fig2.update_layout(title='Distancia entre centros de masas',title_x=0.5)
        fig2.data[0].colorbar.title.text = "Distancia Å"
        fig2.data[0].colorbar.titleside = "right"
        fig2.update_xaxes(title="Cavidad",tickangle=-90)
        fig2.update_yaxes(title="E2012")

        gzr_partes =["E2012","Imidazol","Metoxibenceno","Piperidinona","Metilo","Flurobencilo"]

        viol = go.Figure()
        show_legend = False
        for aa in range(len(aa_min2[0])):
            viol.add_traces(go.Violin(y=al[:,0,aa],
                                     legendgroup='All',scalegroup='All', name='All',
                                     line_color=c2,
                                     meanline_visible=True, showlegend= show_legend,
                                     x0=gzr_partes[0]+" - "+aa_min2[0][aa]))

        viol1 = go.Figure()
        for aa in range(len(aa_min2[1])):
            viol1.add_traces(go.Violin(y=al[:,1,aa],
                                     legendgroup='All',scalegroup='All', name='All',
                                     line_color=c2,
                                     meanline_visible=True, showlegend= show_legend,
                                     x0=gzr_partes[1]+" - "+aa_min2[1][aa]))
        viol2 = go.Figure()
        for aa in range(len(aa_min2[2])):
            viol2.add_traces(go.Violin(y=al[:,2,aa],
                                     legendgroup='All',scalegroup='All', name='All',
                                     line_color=c2,
                                     meanline_visible=True, showlegend= show_legend,
                                     x0=gzr_partes[2]+" - "+aa_min2[2][aa]))
        viol3 = go.Figure()
        for aa in range(len(aa_min2[3])):
            viol3.add_traces(go.Violin(y=al[:,3,aa],
                                     legendgroup='All',scalegroup='All', name='All',
                                     line_color=c2,
                                     meanline_visible=True, showlegend= show_legend,
                                     x0=gzr_partes[3]+" - "+aa_min2[3][aa]))

        viol4 = go.Figure()
        for aa in range(len(aa_min2[4])):
            viol4.add_traces(go.Violin(y=al[:,4,aa],
                                     legendgroup='All',scalegroup='All', name='All',
                                     line_color=c2,
                                     meanline_visible=True, showlegend= show_legend,
                                     x0=gzr_partes[4]+" - "+aa_min2[4][aa]))
        viol5 = go.Figure()
        for aa in range(len(aa_min2[5])):
            viol5.add_traces(go.Violin(y=al[:,5,aa],
                                     legendgroup='All',scalegroup='All', name='All',
                                     line_color=c2,
                                     meanline_visible=True, showlegend= show_legend,
                                     x0=gzr_partes[5]+" - "+aa_min2[5][aa]))
        tit=-1
        for fff in [viol,viol1,viol2,viol3,viol4,viol5]:
            tit+=1
            fff.update_traces(meanline_visible=True,scalemode='width') #scale violin plot area with total count
            fff.update_xaxes(tickangle=-90)
            fff.update_yaxes(title="Distancia (Å)")
            fff.update_layout(title='Distribución de '+gzr_partes[tit],title_x=0.5,
                         violingap=0, violingroupgap=0, violinmode='overlay',yaxis_range=[2,17])
            
        plot= html.Div([
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=fig2,config=f)]),
            ]),
            html.Br(),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=viol,config=f),
                    dcc.Graph(figure=viol2,config=f),
                    dcc.Graph(figure=viol4,config=f),
                ]),
                dbc.Col([
                    dcc.Graph(figure=viol1,config=f),
                    dcc.Graph(figure=viol3,config=f),
                    dcc.Graph(figure=viol5,config=f),
                ])
            ])])
    if ana =="CT":
        if CT =="3A":
            cfc = pd.read_csv("assets/native_contact3series.dat",sep="\s+")
            nn = pd.read_csv("assets/nonnative_contact3series.dat",sep="\s+") 
            dfc = cfc.merge(nn, on='#Frame')
            dfc = dfc.set_index('#Frame')
        elif CT =="4A":
            cfc = pd.read_csv("assets/native_contact4series.dat",sep="\s+")
            nn = pd.read_csv("assets/nonnative_contact4series.dat",sep="\s+") 
            dfc = cfc.merge(nn, on='#Frame')
            dfc = dfc.set_index('#Frame')
        elif CT =="5A":
            cfc = pd.read_csv("assets_viejos/native_contact5series.dat",sep="\s+")
            nn = pd.read_csv("assets_viejos/nonnative_contact5series-002.dat",sep="\s+") 
            dfc = cfc.merge(nn, on='#Frame')
            dfc = dfc.set_index('#Frame')
        elif CT =="6A":
            cfc = pd.read_csv("assets/native_contact6series.dat",sep="\s+")
            nn = pd.read_csv("assets/nonnative_contact6series.dat",sep="\s+") 
            dfc = cfc.merge(nn, on='#Frame')
            dfc = dfc.set_index('#Frame')
        elif CT =="7A":
            cfc = pd.read_csv("assets/native_contact7series.dat",sep="\s+")
            nn = pd.read_csv("assets/nonnative_contact7series.dat",sep="\s+") 
            dfc = cfc.merge(nn, on='#Frame')
            dfc = dfc.set_index('#Frame')

        #Definimos las subpartes de analisis de e2012  y de la cavidad
        imidazol = ["N2_", "C10_", "N3_", "C23_", "C22_","H9_", "H20_", "C25_", "H24_", "H25_", "H26_"]
        metoxi = ["C20_", "C8_", "C19_", "C17_", "C5_", "C9_", "O2_", "C24_", "H21_", "H22_", "H23_", "H19_", "H5_", "H8_"]
        piper = ["C14_", "C15_", "N1_", "C11_", "C12_", "C1_", "O1_", "H10_", "H11_", "H12_", "H13_", "H1_", "H2_", "C3_", "H3_", "C13_", "H14_"]
        metil = ["C13_", "H14_", "C16_", "H15_", "H16_", "H17_"]
        fluoro = ["C2_", "C4_", "C6_", "C21_", "C7_", "C18_", "H4_", "H6_", "F1_", "H7_", "H18_"]
        partes_e2012 = [imidazol,metoxi,piper,metil,fluoro]
        cav= ["210","211","701", "702", "706", "707","708","709","710","711","712","713", "769", "772","773","774", "776", "777", "780", "827","828","829","830","831","832","833", "835","836","1332","1333","1334","1335", "1338","1339","1340", "1342","1343","1344","1345","1346"]
        cav_chido =['NCST-ILE-242', 'NCST-ASN-243', 'PSEN-PHE-105', 'PSEN-TYR-106', 'PSEN-ASP-110', 'PSEN-GLY-111','PSEN-GLN-112', 'PSEN-LEU-113', 'PSEN-ILE-114', 'PSEN-TYR-115', 'PSEN-THR-116', 'PSEN-PRO-117','PSEN-LEU-173', 'PSEN-PHE-176', 'PSEN-PHE-177', 'PSEN-SER-178', 'PSEN-ILE-180', 'PSEN-TYR-181','PSEN-GLU-184', 'PSEN-ALA-231', 'PSEN-LEU-232', 'PSEN-MET-233', 'PSEN-ALA-234', 'PSEN-LEU-235','PSEN-VAL-236', 'PSEN-PHE-237', 'PSEN-LYS-239', 'PSEN-TYR-240','AB-GLY-25', 'AB-SER-26','AB-ASN-27', 'AB-LYS-28', 'AB-ILE-31', 'AB-ILE-32','AB-GLY-33', 'AB-MET-35', 'AB-VAL-36', 'AB-GLY-37', 'AB-GLY-38','AB-VAL-39']

        #Primero vamos a separar el dfc de contactos en cada una de las subpartes
        cols = [[] for i in partes_e2012]
        c = -1
        for partes in partes_e2012:
            c+=1
            for atom in partes:
                for  col in dfc.columns.values:
                    if atom in col:
                        cols[c].append(col)
        #Ahora vamos a separar por subpartes y por cada aminoacido, asi podemos ver con quien si esta interactuando
        cols_amino = [[[] for i in cav] for j in partes_e2012]
        c = -1
        for partes in cols:              #Para cada parte en e2012
            c+=1
            c2=-1
            for aa in cav:               #checar cada aminoacido
                c2+=1
                for atom in partes:     #si hay un contacto o no 
                    if aa in atom:
                        cols_amino[c][c2].append(atom)

        #Hagamos la sumas de los contactos
        contactos = [[[] for i in cav] for j in partes_e2012]
        c = -1
        for i in  cols_amino:
            c+=1
            c2=-1
            for amino in i:
                c2+=1
                if len(amino)==0:
                    contactos[c][c2].append(0)
                else:
                    contactos[c][c2].append(np.mean(np.sum(dfc[cols_amino[c][c2]].values, axis=1)/len(cols_amino[c][c2])))
        contactos = np.array(contactos).reshape(5,40)

        fig2 = go.Figure()
        fig2.add_traces(go.Heatmap(z=contactos.flatten(),
                                  x=cav_chido*5,
                                  y=np.array([[i]*len(cav) for i in["Imidazol","Metoxibenceno","Piperidona","Metilo","Fluorobencilo"]]).flatten(),
                                  colorscale='OrRd'))

        fig2.update_layout(title='Matriz de contactos <br> E2012-Cavidad',title_x=0.5)
        fig2.data[0].colorbar.title.text = "Núm. contactos normalizados"
        fig2.data[0].colorbar.titleside = "right"
        fig2.update_xaxes(title="Cavidad",tickangle=-90)
        fig2.update_yaxes(title="E2012")
        plot= html.Div([
            html.Br(),
            html.Br(),
            html.Span("Esta matriz se obtuvo observando el número de contactos (nativos y no nativos) en cada frame de la trayectoria. Se muestra el promedio de contactos por frame entre contactos posibles."),
            dbc.Row([
                dbc.Col([
                    dcc.Graph(figure=fig2,config=f)])]),
            
            html.Hr(),
            
            dbc.Row(
            dbc.Col(
                html.Img(src="assets/e2012.png",style={'height':'120%', 'width':'80%'}),
                width={"size": 6, "offset": 3},
            )
        ),
        ])
    if ana =="HB":
        hb = pd.read_csv("assets/nhb.dat",sep="\s+") 
        hb["Ang"] = np.genfromtxt("assets/angle_hb.dat")[:,1]
        hb["Dist"] = np.genfromtxt("assets/dist_hb.dat")[:,1]
        frac= np.round(hb["HB_00060[UU]"].sum()/hb.shape[0],4)*100
        
        fih = go.Figure()
        fih.add_traces(go.Violin(y=hb[hb["HB_00060[UU]"]==1]["Ang"].values,x=["Ángulo D-H-A"]*hb["HB_00060[UU]"].sum(),
                                 line_color="teal",
                                 meanline_visible=True,))
        fih.update_layout(title='Puente de hidrogeno E2012-Tyr106',title_x=0.5)
        fih.update_yaxes(title="Ángulo (°)")
        
        fih2 = go.Figure()
        fih2.add_traces(go.Violin(y=hb[hb["HB_00060[UU]"]==1]["Dist"].values,x=["Distancia Aceptor-Donador"]*hb["HB_00060[UU]"].sum(),
                                 line_color="teal",
                                 meanline_visible=True,))
        fih2.update_layout(title='Puente de hidrogeno E2012-Tyr106',title_x=0.5)
        fih2.update_yaxes(title="Distancia (Å)")
        plot= html.Div([
            dbc.Row([
                html.Br(),

                html.Br(),
                html.Span("Este puente de hidrogeno esta prestente en el "+str(frac)+"% de la trayectoria. Lo podemos clasificar de acuerdo a sus caracterissticas con una fuerza Media."),
                dbc.Col([
                    dcc.Graph(figure=fih,config=f)]),
                dbc.Col([
                    dcc.Graph(figure=fih2,config=f)]),
            ])])
    return plot

##  Analisis ligandos

In [27]:
#********************************A N A L I S I S  D E  L I G A N D O S*****************************************************   
@app.callback(
    Output("ligplot", "children"),
    Input(component_id="liganal", component_property='value'),
    Input(component_id="var1", component_property='value'),
    Input(component_id="var2", component_property='value'),
    Input(component_id="color", component_property='value'),
    Input(component_id="graph", component_property='value'),
    Input(component_id="img-format", component_property='data'),
)
def ligbuilder(ana,var1,var2,color,graph,f):
    df = pd.read_csv("/home/chemachiles/Downloads/gs_analisis/assets/PERMM.csv")
    df2 = pd.read_csv("/home/chemachiles/Downloads/gs_analisis/assets/ligbuilder_grande_swiss+permm.csv")
    if ana =="Umb":
        e_e2012 =  np.array([float(i) for i in E[E["name"]=="CHEMBL1224151"]["energia"].values[0][1:-1].split(", ")])
        e_global = [0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,-0.03,-0.11,-0.28,-0.56,-0.95,-1.59,-2.09,-2.37,-2.66,-2.95,-3.07,-3.31,-3.49,-3.68,-3.80,-4.13,-4.63,-5.47,-6.03,-6.14,-6.06,-5.88,-5.91,-5.96,-5.89,-5.25,-4.39,-4.27,-4.30,-4.27,-4.21,-4.13,-4.04,-3.78,-4.03,-4.13,-4.20,-4.26,-4.29,-4.27,-4.38,-5.22,-5.88, -5.97,-5.91,-5.88,-6.06,-6.13,-6.04,-5.52,-4.66,-4.15,-3.80,-3.68,-3.50,-3.32,-3.08,-2.95,-2.68,-2.38,-2.11,-1.61,-0.97,-0.57,-0.29,-0.12,-0.04,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00]
        fig=go.Figure()
        fig.add_traces(go.Scatter(x=umbrella[:,0],y=umbrella[:,1],name="Umbrella POPC fenilimidazol"))
        # fig.add_traces(go.Scatter(x=umbrelladopc[:,0],y=umbrelladopc[:,1],name="Umbrella DOPC"))
        fig.add_traces(go.Scatter(x=[i for i in range(-45,45)],y=e_e2012,name="PERMM E2012 drag"))
        fig.add_traces(go.Scatter(x=[i for i in range(-45,45)],y=e_global,name="PERMM E2012 global"))
        fig.add_traces(go.Scatter(x=permm_imi["Z"],y=permm_imi["Fenilimidazol"],name="PERMM fenilimidazol"))

        fig.add_vrect(x0=np.mean(pos_gzr)-np.std(pos_gzr), x1=np.mean(pos_gzr)+np.std(pos_gzr), 
                      annotation_text="COM", annotation_position="top",
                      fillcolor="teal", opacity=0.25, line_width=0)
        
        fig.add_vrect(x0=np.mean(pos_gzr)*-1-np.std(pos_gzr), x1=np.mean(pos_gzr)*-1+np.std(pos_gzr), 
                      annotation_text="COM", annotation_position="top",
                      fillcolor="teal", opacity=0.25, line_width=0)
        
        fig.add_vline(x=np.mean(pos_gzr), line_width=3, line_dash="dash", line_color="blue")
        fig.add_vline(x=np.mean(pos_gzr)*-1, line_width=3, line_dash="dash", line_color="blue")
        
        fig.update_layout(title="Perfl energetico de E2012 en POPC",title_x=0.5)
        fig.update_xaxes(title="Distancia del centro de la membrana en Å")
        fig.update_yaxes(title="kcal/mol")
        plot= html.Div([
            dbc.Row([
                html.Br(),
                dbc.Col([
                    dcc.Graph(figure=fig,config=f)]),
            ])])
    elif ana=="2var":
        if graph=="scatter":
            fig = px.scatter(todo,x=var1,y=var2,color=color, symbol="lig_gr",hover_name="pdb")
        elif graph=="violin":
            fig = px.violin(todo,x=var1,y=var2,color=color,hover_name="pdb", points="all", box=True,)
            
        plot= html.Div([
            dcc.Graph(figure=fig,config=f),
            ])
        
    elif ana=="LogP":
        fig1 = go.Figure()
        fig1.add_trace(go.Scatter(x=df[df["Log of perm. coeff. - BLM"]>0]["CLASE"],
                                 y=df[df["Log of perm. coeff. - BLM"]>0]["Log of perm. coeff. - BLM"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[df["Log of perm. coeff. - BLM"]>0]["name"]))

        fig1.add_trace(go.Scatter(x=df[df["Log of perm. coeff. - BLM"]<0]["CLASE"],
                                 y=df[df["Log of perm. coeff. - BLM"]<0]["Log of perm. coeff. - BLM"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df[df["Log of perm. coeff. - BLM"]<0]["name"]))
        
        fig1.add_trace(go.Scatter(x=df2[df2["Log of perm. coeff. - BLM"]<0]["CLASE"],
                                 y=df2[df2["Log of perm. coeff. - BLM"]<0]["Log of perm. coeff. - BLM"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df2[df2["Log of perm. coeff. - BLM"]<0]["name"]))
        
        fig1.update_layout(title="Distribución de ligandos de GS",title_x=0.5,yaxis_title="Log of perm. coeff. - BLM")

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(x=df[df["Log of perm. coeff - BBB (Po)"]>-4.35]["CLASE"],
                                 y=df[df["Log of perm. coeff - BBB (Po)"]>-4.35]["Log of perm. coeff - BBB (Po)"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[df["Log of perm. coeff - BBB (Po)"]>-4.35]["name"]))

        fig2.add_trace(go.Scatter(x=df[df["Log of perm. coeff - BBB (Po)"]<-4.35]["CLASE"],
                                 y=df[df["Log of perm. coeff - BBB (Po)"]<-4.35]["Log of perm. coeff - BBB (Po)"],
                                 mode='markers',marker_color="red",name="No permeable",
                                text=df[df["Log of perm. coeff - BBB (Po)"]<-4.35]["name"]))
        
        fig2.add_trace(go.Scatter(x=df2[df2["Log of perm. coeff - BBB (Po)"]>-4.35]["CLASE"],
                                 y=df2[df2["Log of perm. coeff - BBB (Po)"]>-4.35]["Log of perm. coeff - BBB (Po)"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df2[df2["Log of perm. coeff - BBB (Po)"]>-4.35]["name"]))
        
        fig2.add_trace(go.Scatter(x=df2[df2["Log of perm. coeff - BBB (Po)"]<-4.35]["CLASE"],
                                 y=df2[df2["Log of perm. coeff - BBB (Po)"]<-4.35]["Log of perm. coeff - BBB (Po)"],
                                 mode='markers',marker_color="red",name="No permeable",
                                text=df[df["Log of perm. coeff - BBB (Po)"]<-4.35]["name"]))
        fig2.update_layout(title="Distribución de ligandos de GS",title_x=0.5,yaxis_title="Log of perm. coeff - BBB (Po)")

        fig3 = go.Figure()
        fig3.add_trace(go.Scatter(x=df[df["Log of perm. coeff - CACO2 (Po)"]>0]["CLASE"],
                                 y=df[df["Log of perm. coeff - CACO2 (Po)"]>0]["Log of perm. coeff - CACO2 (Po)"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[df["Log of perm. coeff - CACO2 (Po)"]>0]["name"]))

        fig3.add_trace(go.Scatter(x=df[df["Log of perm. coeff - CACO2 (Po)"]<0]["CLASE"],
                                 y=df[df["Log of perm. coeff - CACO2 (Po)"]<0]["Log of perm. coeff - CACO2 (Po)"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df[df["Log of perm. coeff - CACO2 (Po)"]<0]["name"]))
        fig3.add_trace(go.Scatter(x=df2[df2["Log of perm. coeff - CACO2 (Po)"]<0]["CLASE"],
                                 y=df2[df2["Log of perm. coeff - CACO2 (Po)"]<0]["Log of perm. coeff - CACO2 (Po)"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df2[df2["Log of perm. coeff - CACO2 (Po)"]<0]["name"]))
        fig3.update_layout(title="Distribución de ligandos de GS",title_x=0.5,yaxis_title="Log of perm. coeff - CACO2 (Po)")


        fig4 = go.Figure()
        fig4.add_trace(go.Scatter(x=df[df["Log of perm. coeff - PAMPA-DS (Po)"]>0]["CLASE"],
                                 y=df[df["Log of perm. coeff - PAMPA-DS (Po)"]>0]["Log of perm. coeff - PAMPA-DS (Po)"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[df["Log of perm. coeff - PAMPA-DS (Po)"]>0]["name"]))

        fig4.add_trace(go.Scatter(x=df[df["Log of perm. coeff - PAMPA-DS (Po)"]<0]["CLASE"],
                                 y=df[df["Log of perm. coeff - PAMPA-DS (Po)"]<0]["Log of perm. coeff - PAMPA-DS (Po)"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df[df["Log of perm. coeff - PAMPA-DS (Po)"]<0]["name"]))
        fig4.add_trace(go.Scatter(x=df2[df2["Log of perm. coeff - PAMPA-DS (Po)"]<0]["CLASE"],
                                 y=df2[df2["Log of perm. coeff - PAMPA-DS (Po)"]<0]["Log of perm. coeff - PAMPA-DS (Po)"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df2[df2["Log of perm. coeff - PAMPA-DS (Po)"]<0]["name"]))
        fig4.update_layout(title="Distribución de ligandos de GS",title_x=0.5,yaxis_title="Log of perm. coeff - PAMPA-DS (Po)")


        fig5 = go.Figure()
        fig5.add_trace(go.Scatter(x=df[df["Log of perm. coeff - plasma membr"]>0]["CLASE"],
                                 y=df[df["Log of perm. coeff - plasma membr"]>0]["Log of perm. coeff - plasma membr"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[df["Log of perm. coeff - plasma membr"]>0]["name"]))

        fig5.add_trace(go.Scatter(x=df[df["Log of perm. coeff - plasma membr"]<0]["CLASE"],
                                 y=df[df["Log of perm. coeff - plasma membr"]<0]["Log of perm. coeff - plasma membr"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df[df["Log of perm. coeff - plasma membr"]<0]["name"]))
        fig5.add_trace(go.Scatter(x=df2[df2["Log of perm. coeff - plasma membr"]<0]["CLASE"],
                                 y=df2[df2["Log of perm. coeff - plasma membr"]<0]["Log of perm. coeff - plasma membr"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df2[df2["Log of perm. coeff - plasma membr"]<0]["name"]))
        fig5.update_layout(title="Distribución de ligandos de GS",title_x=0.5,yaxis_title="Log of perm. coeff - plasma membr")

        plot = html.Div([
                    dcc.Graph(figure=fig1,config=f),
                    dcc.Graph(figure=fig3,config=f),
                    dcc.Graph(figure=fig2,config=f),
                    dcc.Graph(figure=fig4,config=f),
                    dcc.Graph(figure=fig5,config=f),
                    html.Br(),
                    html.Hr(),
                    dbc.Row([
                        dbc.Col([
                            html.B("Cavidad con cluster 81"),
                            html.Img(src="assets/cluster81_cav.png",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center")
                        ], width=5),
                        dbc.Col([
                            html.B("Cavidad con E2012"),
                            html.Img(src="assets/e2012.png",style={'height':'90%', 'width':'90%'},className="d-flex align-self-center")
                        ], width=5),
                    ]),
                    html.Hr(),
                    html.Img(src="assets/Cluster81.png")
                ])
    elif ana=="IC50":
        fig1 = go.Figure()
        fig1.add_trace(go.Scatter(y=df[(df["Log of perm. coeff. - BLM"]>0) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff. - BLM"]>0) & (P["ID_CLASE"]==10)]["Log of perm. coeff. - BLM"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[(df["Log of perm. coeff. - BLM"]>0) & (P["ID_CLASE"]==10)]["name"]))

        fig1.add_trace(go.Scatter(y=df[(df["Log of perm. coeff. - BLM"]<0) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff. - BLM"]<0) & (P["ID_CLASE"]==10)]["Log of perm. coeff. - BLM"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df[(df["Log of perm. coeff. - BLM"]<0) & (P["ID_CLASE"]==10)]["name"]))
        fig1.update_layout(title="Distribución de ligandos de GS",title_x=0.5,xaxis_title="Log of perm. coeff. - BLM",yaxis_title="IC50")

        fig2 = go.Figure()
        fig2.add_trace(go.Scatter(y=df[(df["Log of perm. coeff - BBB (Po)"]>-4.35) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff - BBB (Po)"]>-4.35) & (P["ID_CLASE"]==10)]["Log of perm. coeff - BBB (Po)"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[(df["Log of perm. coeff - BBB (Po)"]>-4.35) & (P["ID_CLASE"]==10)]["name"]))

        fig2.add_trace(go.Scatter(y=df[(df["Log of perm. coeff - BBB (Po)"]<-4.35) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff - BBB (Po)"]<-4.35) & (P["ID_CLASE"]==10)]["Log of perm. coeff - BBB (Po)"],
                                 mode='markers',marker_color="red",name="No permeable",
                                text=df[(df["Log of perm. coeff - BBB (Po)"]<-4.35) & (P["ID_CLASE"]==10)]["name"]))
        fig2.update_layout(title="Distribución de ligandos de GS",title_x=0.5,xaxis_title="Log of perm. coeff - BBB (Po)",yaxis_title="IC50")

        fig3 = go.Figure()
        fig3.add_trace(go.Scatter(y=df[(df["Log of perm. coeff - CACO2 (Po)"]>0) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff - CACO2 (Po)"]>0) & (P["ID_CLASE"]==10)]["Log of perm. coeff - CACO2 (Po)"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[(df["Log of perm. coeff - CACO2 (Po)"]>0) & (P["ID_CLASE"]==10)]["name"]))

        fig3.add_trace(go.Scatter(y=df[(df["Log of perm. coeff - CACO2 (Po)"]<0) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff - CACO2 (Po)"]<0) & (P["ID_CLASE"]==10)]["Log of perm. coeff - CACO2 (Po)"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df[(df["Log of perm. coeff - CACO2 (Po)"]<0) & (P["ID_CLASE"]==10)]["name"]))
        fig3.update_layout(title="Distribución de ligandos de GS",title_x=0.5,xaxis_title="Log of perm. coeff - CACO2 (Po)",yaxis_title="IC50")


        fig4 = go.Figure()
        fig4.add_trace(go.Scatter(y=df[(df["Log of perm. coeff - PAMPA-DS (Po)"]>0) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff - PAMPA-DS (Po)"]>0) & (P["ID_CLASE"]==10)]["Log of perm. coeff - PAMPA-DS (Po)"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[(df["Log of perm. coeff - PAMPA-DS (Po)"]>0) & (P["ID_CLASE"]==10)]["name"]))

        fig4.add_trace(go.Scatter(y=df[(df["Log of perm. coeff - PAMPA-DS (Po)"]<0) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff - PAMPA-DS (Po)"]<0) & (P["ID_CLASE"]==10)]["Log of perm. coeff - PAMPA-DS (Po)"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df[(df["Log of perm. coeff - PAMPA-DS (Po)"]<0) & (P["ID_CLASE"]==10)]["name"]))
        fig4.update_layout(title="Distribución de ligandos de GS",title_x=0.5,xaxis_title="Log of perm. coeff - PAMPA-DS (Po)",yaxis_title="IC50")


        fig5 = go.Figure()
        fig5.add_trace(go.Scatter(y=df[(df["Log of perm. coeff - plasma membr"]>0) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff - plasma membr"]>0) & (P["ID_CLASE"]==10)]["Log of perm. coeff - plasma membr"],
                                 mode='markers',marker_color="blue",name="Permeable",
                                text=df[(df["Log of perm. coeff - plasma membr"]>0) & (P["ID_CLASE"]==10)]["name"]))

        fig5.add_trace(go.Scatter(y=df[(df["Log of perm. coeff - plasma membr"]<0) & (P["ID_CLASE"]==10)]["PIC50_VALUE"],
                                 x=df[(df["Log of perm. coeff - plasma membr"]<0) & (P["ID_CLASE"]==10)]["Log of perm. coeff - plasma membr"],
                                 mode='markers',marker_color="red",name="Impermeable",
                                text=df[(df["Log of perm. coeff - plasma membr"]<0) & (P["ID_CLASE"]==10)]["name"]))
        fig5.update_layout(title="Distribución de ligandos de GS",title_x=0.5,xaxis_title="Log of perm. coeff - plasma membr",yaxis_title="IC50")
        swiss = pd.read_csv("assets/swiss.csv")
        fig6 = go.Figure()
        fig6.add_trace(go.Scatter(x=swiss["XLOGP3"],
                                 y=swiss["pIC50"],
                                 mode='markers',showlegend=False,
                                 text=swiss["Molecule"],
                                 marker=dict(
                                        size=7,
                                        color=swiss["ligand-lipophilicity efficiency (LLE)"],
                                        colorscale='Viridis',  # Puedes cambiar la paleta de colores aquí
                                        colorbar=dict(title='LLE',titleside="right"))))

        fig6.update_layout(title="Ligand-lipophilicity efficiency (LLE)",title_x=0.5,yaxis_title="pIC50",xaxis_title="XlogP3")
        plot = html.Div([
                    dcc.Graph(figure=fig1,config=f),
                    dcc.Graph(figure=fig3,config=f),
                    dcc.Graph(figure=fig2,config=f),
                    dcc.Graph(figure=fig4,config=f),
                    dcc.Graph(figure=fig5,config=f),
                    dcc.Graph(figure=fig6,config=f)])
    return plot

In [ ]:
server = app.server
# =============================================================================
if __name__ == '__main__':               #Dash se basa un poco en flask por eso esa sintaxis
    app.run_server(port = 1987) 
# =============================================================================
# if __name__ == '__main__':               #Dash se basa un poco en flask por eso esa sintaxis
#     app.run_server(debug=True) 

Dash is running on http://127.0.0.1:1987/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:1987
Press CTRL+C to quit
127.0.0.1 - - [15/May/2024 10:33:22] "GET /e2012 HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 10:33:22] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 10:33:22] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 10:33:22] "GET /_favicon.ico?v=2.7.0 HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 10:33:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 10:33:22] "GET /_dash-component-suites/dash/dcc/async-markdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 10:33:22] "GET /_dash-component-suites/dash/dcc/async-dropdown.js HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 10:33:22] "GET /assets/mdcbq.svg HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 10:33:22] "POST /_dash-update-component HTTP/1.1" 200 -
127.0.0.1 - - [15/May/2024 10:33:22] "GET /assets/UNAM.png HTTP/1.1" 304 -
127.0.0.1 - - [15/May/2024 10:33:22] "GET /_dash-component-suites/dash/dcc/async-highlight.js HTTP/1.1" 304 -


# Fin